In [ ]:
import hashlib
import io
import pathlib

In [ ]:
import unittest
self = unittest.TestCase()

In [ ]:
datap = pathlib.Path('data')

In [ ]:
data = datap.read_bytes()

In [ ]:
data[:4].hex()

In [ ]:
NSIS_COMPRESS_BZIP2_LEVEL = 9
BZ_G_SIZE = 50
BZ_MAX_SELECTORS = int(2 + (900000 / BZ_G_SIZE))
BZ_N_GROUPS = 6
BZ_MAX_ALPHA_SIZE = 258
BZ_MAX_CODE_LEN = 23
MTFA_SIZE = 4096
MTFL_SIZE = 16
BZ_RUNA = 0
BZ_RUNB = 1

In [ ]:
class UnRLE:
    """Remove run length encoding from compressed data."""
    def __init__(self, avail_out, tt, nblock, nblock_used, k0, tpos):
        self.out_ch = 0
        self.out_len = 0

        self.nblock_used = nblock_used
        self.k0 = k0
        self.tpos = tpos
        self.tt = tt
        self.nblockpp = nblock + 1

        self.avail_out = avail_out
        self.next_out = bytearray()

        self.k1 = None

    def _op1(self):
        if self.avail_out == 0:
            return True
        if self.out_len == 1:
            return False
        self.next_out.append(self.out_ch)
        self.out_len -= 1
        self.avail_out -= 1
    
    def _op2(self):
        if self.avail_out == 0:
            self.out_len = 1
            return True
        self.next_out.append(self.out_ch)
        self.avail_out -= 1
    
    def _op3(self):
        if self.nblock_used == self.nblockpp:
            self.out_len = 0
            return True
        self.out_ch = self.k0
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
    
    def _op4(self):
        if self._op2():
            return True
        if self._op3():
            return True
        if self.k1 != self.k0:
            self.k0 = self.k1
            if self._op4():
                return True
            return False
        if self.nblock_used == self.nblockpp:
            if self._op4():
                return True
            return False
    
        self._op5()
    
    def _op5(self):
        self.out_len = 2
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        if self.nblock_used == self.nblockpp:
            return
        if self.k1 != self.k0:
            self.k0 = self.k1
            return
    
        self.out_len = 3
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        if self.nblock_used == self.nblockpp:
            return
        if self.k1 != self.k0:
            self.k0 = self.k1
            return
    
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        self.out_len = self.k1 + 4
        self.tpos = self.tt[self.tpos]
        self.k0 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1

    def run(self):
        while True:
            if self.out_len > 0:
                while True:
                    match self._op1():
                        case False:
                            break
                        case True:
                            return self.next_out
                if self._op2():
                    return self.next_out
            if self._op3():
                return self.next_out
            if self.k1 != self.k0:
                self.k0 = self.k1
                if self._op4():
                    return self.next_out
                continue
            if self.nblock_used == self.nblockpp:
                if self._op4():
                    return self.next_out
                continue
        
            self._op5()

        return self.next_out

In [ ]:
class Bz2Decompress:
    def __init__(self, data, avail_out):
        if not isinstance(data, bytes):
            raise TypeError('Input must be bytes')
        self.f = io.BytesIO(data)
        self.avail_out = avail_out

        self.bslive = 0
        self.bsbuff = 0

        self.origptr = None

        self.ninuse = None
        self.seqtounseq = None
        self.alphasize = None

        self.ngroups = None
        self.nselectors = None
        self.selector = None

        self.len = None

        self.perm = None
        self.base = None
        self.limit = None
        self.minlens = None

        self.eob = None
        self.nblockmax = None
        self.groupno = None
        self.grouppos = None
        self.unzftab = None
        self.nblock = None

        self.mtfa = None
        self.mtfbase = None

        self.nextsym = None

        self.tt = None

        self.k0 = None
        self.tpos = None
        self.nblock_used = None

        self.out = None

    def _get_bits(self, n, err=None):
        """Read n bits from the compressed data."""
        while True:
            if self.bslive >= n:
                v = (self.bsbuff >> (self.bslive - n)) & ((1 << n) - 1)
                self.bslive -= n
    
                return v
    
            next_in = self.f.read(1)
            if not len(next_in):
                raise RuntimeError(f'BZ_DATA_ERROR: {err} not found')

            self.bsbuff = (self.bsbuff << 8) | int.from_bytes(next_in)
            self.bslive += 8

    def _block_header(self):
        """Check that the compressed stream starts with the correct abbreviated NSIS Bzip2 block header."""
        uc = self._get_bits(8, 'BZ_X_BLKHDR_1')

        if uc != 0x31:
            raise RuntimeError('BZ_DATA_ERROR: Block header BZ_X_BLKHDR_1 not found')

    def _origptr(self):
        """Construct the origin pointer."""
        origptr = 0
        uc = self._get_bits(8, 'BZ_X_ORIGPTR_1')
        origptr = (origptr << 8) | uc
        uc = self._get_bits(8, 'BZ_X_ORIGPTR_2')
        origptr = (origptr << 8) | uc
        uc = self._get_bits(8, 'BZ_X_ORIGPTR_3')
        origptr = (origptr << 8) | uc

        if origptr < 0:
            raise RuntimeError('BZ_DATA_ERROR: Origin pointer is negative')
        if origptr > 10 + NSIS_COMPRESS_BZIP2_LEVEL * 100000:
            raise RuntimeError('BZ_DATA_ERROR: Origin pointer is too large')

        self.origptr = origptr

    def _mapping_table(self):
        """Receive the mapping table."""
        inuse16 = list()
        for i in range(16):
            uc = self._get_bits(1, 'BZ_X_MAPPING_1')
            if uc == 1:
                inuse16.append(True)
            else:
                inuse16.append(False)

        inuse = [False] * 256
        for i in range(16):
            if inuse16[i]:
                for j in range(16):
                    uc = bzd._get_bits(1, 'BZ_X_MAPPING_2')
                    if uc == 1:
                        inuse[i * 16 + j] = True
        
        ninuse = 0
        seqtounseq = [0] * 256
        for qi in range(256):
            if inuse[qi]:
                seqtounseq[ninuse] = qi
                ninuse += 1
        
        if ninuse == 0:
            raise RuntimeError('BZ_DATA_ERROR: ninuse is zero')

        self.ninuse = ninuse
        self.seqtounseq = seqtounseq
        self.alphasize = ninuse + 2

    def _selectors(self):
        """Get the selectors."""
        ngroups = self._get_bits(3, 'BZ_X_SELECTOR_1')
        if ngroups < 2 or ngroups > 6:
            raise RuntimeError('BZ_DATA_ERROR: Invalid number of groups')

        nselectors = self._get_bits(15, 'BZ_X_SELECTOR_2')
        if nselectors < 1:  # Maybe this can be nselectors == 0 ?
            raise RuntimeError('BZ_DATA_ERROR: Invalid number of selectors')

        selectormtf = [0] * BZ_MAX_SELECTORS
        for i in range(nselectors):
            j = 0
            while True:
                uc = self._get_bits(1, 'BZ_X_SELECTOR_3')
                if uc == 0:  # Why not store this value as 3 bits rather than a set of bits? It can only be the number of groups which is already stored as 3 bits above.
                    break
                j += 1
                if j >= ngroups:
                    raise RuntimeError('BZ_DATA_ERROR: Invalid selectorMtf value')
            selectormtf[i] = j

        # Undo the MTF values for the selectors.
        pos = [0] * BZ_N_GROUPS
        
        for i in range(ngroups):
            pos[i] = i
        
        selector = [0] * BZ_MAX_SELECTORS
        
        for i in range(nselectors):
            v = selectormtf[i]
            tmp = pos[v]
            while v > 0:
                pos[v] = pos[v-1]
                v -= 1
            pos[0] = tmp
            selector[i] = tmp
        
        self.ngroups = ngroups
        self.nselectors = nselectors
        self.selector = selector

    def _coding_tables(self):
        """Get the coding tables."""
        lenn = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]
        for i in range(self.ngroups):
            curr = self._get_bits(5, 'BZ_X_CODING_1')
            for j in range(self.alphasize):
                while True:
                    if curr < 1 or curr > 20:
                        raise RuntimeError('BZ_DATA_ERROR: Value curr outside of acceptable range')
                    uc = self._get_bits(1, 'BZ_X_CODING_2')
                    if uc == 0:
                        break
                    uc = self._get_bits(1, 'BZ_X_CODING_3')
                    if uc == 0:
                        curr += 1
                    else:
                        curr -= 1
                lenn[i][j] = curr

        self.len = lenn

    def _huffman_tables(self):
        """Create the Huffman decoding tables."""
        l = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]
        b = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]
        p = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]

        minlens = [0] * BZ_N_GROUPS
        
        for t in range(self.ngroups):
            minlen = 32
            maxlen = 8
            for i in range(self.alphasize):
                if self.len[t][i] > maxlen:
                    maxlen = self.len[t][i]
                if self.len[t][i] < minlen:
                    minlen = self.len[t][i]
        
            limit = l[t]
            base = b[t]
            perm = p[t]
            length = self.len[t]
        
            pp = 0
            for i in range(minlen, maxlen + 1):
                for j in range(self.alphasize):
                    if length[j] == i:
                        perm[pp] = j
                        pp += 1
        
            for i in range(BZ_MAX_CODE_LEN):
                base[i] = 0
        
            for i in range(self.alphasize):
                base[length[i] + 1] += 1
        
            for i in range(BZ_MAX_CODE_LEN):
                base[i] += base[i - 1]
        
            for i in range(BZ_MAX_CODE_LEN):
                limit[i] = 0
        
            vec = 0
            for i in range(minlen, maxlen + 1):
                vec += base[i + 1] - base[i]
                limit[i] = vec - 1
                vec <<= 1
        
            for i in range(minlen + 1, maxlen + 1):
                base[i] = ((limit[i - 1] + 1) << 1) - base[i]
        
            minlens[t] = minlen

        self.limit = l
        self.base = b
        self.perm = p

        self.minlens = minlens

    def _mtf_values(self):
        """Set MTF values."""
        self.eob = self.ninuse + 1
        self.nblockmax = NSIS_COMPRESS_BZIP2_LEVEL * 100_000
        self.groupno = -1
        self.grouppos = 0

        self.unzftab = [0] * 256

        self.nblock = 0

    def _mtf_init(self):
        """Initialize the move to front lists."""
        mtfa = [0] * MTFA_SIZE
        mtfbase = [0] * int(256 / MTFL_SIZE)

        k = MTFA_SIZE - 1
        
        for i in reversed(range(int(256 / MTFL_SIZE))):
            for j in reversed(range(MTFL_SIZE)):
                mtfa[k] = i * MTFL_SIZE + j
                k -= 1
            mtfbase[i] = k + 1

        self.mtfa = mtfa
        self.mtfbase = mtfbase

    def _first_mtf(self):
        """Perform first move to front cycle."""
        gsel = 0
        gminlen = 0
        glimit = 0
        gperm = 0
        gbase = 0
        zn = 0

        if self.grouppos == 0:
            self.groupno += 1
            if self.groupno >= self.nselectors:
                raise RuntimeError('BZ_DATA_ERROR: Number of groups larger than number of selectors')
            self.grouppos = BZ_G_SIZE
            gsel = self.selector[self.groupno]
            gminlen = self.minlens[gsel]
            glimit = self.limit[gsel]
            gperm = self.perm[gsel]
            gbase = self.base[gsel]
        self.grouppos -= 1
        zn = gminlen
        
        zvec = self._get_bits(zn, 'BZ_X_MTF_1')
        
        while True:
            if zn > 20:
                raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
            if zvec <= glimit[zn]:
                break
            zn += 1
            zj = self._get_bits(1, 'BZ_X_MTF_2')
            zvec = (zvec << 1) | zj
        
        if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
            raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')
        
        nextsym = gperm[zvec - gbase[zn]]

        self.gsel = gsel
        self.gminlen = gminlen
        self.glimit = glimit
        self.gperm = gperm
        self.gbase = gbase
        self.nextsym = nextsym

    def _decompress(self):
        """Perform the remaining decompression phases."""
        tt = [0] * NSIS_COMPRESS_BZIP2_LEVEL * 100000

        while True:
            if self.nextsym == self.eob:
                break
        
            if self.nextsym == BZ_RUNA or self.nextsym == BZ_RUNB:
                es = -1
                n = 1
        
                while self.nextsym == BZ_RUNA or self.nextsym == BZ_RUNB:
                    if self.nextsym == BZ_RUNA:
                        es += n
                    n = n << 1
                    if self.nextsym == BZ_RUNB:
                        es += n
        
                    if self.grouppos == 0:
                        self.groupno += 1
                        if self.groupno >= self.nselectors:
                            raise RuntimeError('BZ_DATA_ERROR: Number of groups larger than number of selectors')
                        self.grouppos = BZ_G_SIZE
                        self.gsel = self.selector[self.groupno]
                        self.gminlen = self.minlens[self.gsel]
                        self.glimit = self.limit[self.gsel]
                        self.gperm = self.perm[self.gsel]
                        self.gbase = self.base[self.gsel]
                    self.grouppos -= 1
                    zn = self.gminlen
        
                    zvec = self._get_bits(zn, 'BZ_X_MTF_3')
        
                    while True:
                        if zn > 20:
                            raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
                        if zvec <= self.glimit[zn]:
                            break
                        zn += 1
                        zj = self._get_bits(1, 'BZ_X_MTF_4')
                        zvec = (zvec << 1) | zj
        
                    if zvec - self.gbase[zn] < 0 or zvec - self.gbase[zn] >= BZ_MAX_ALPHA_SIZE:
                        raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')
                    self.nextsym = self.gperm[zvec - self.gbase[zn]]
        
                es += 1
                uc = self.seqtounseq[self.mtfa[self.mtfbase[0]]]
                self.unzftab[uc] += es
        
                while es > 0:
                    if self.nblock >= self.nblockmax:
                        raise RuntimeError('BZ_DATA_ERROR: Value nblock too large')
                    tt[self.nblock] = uc
                    self.nblock += 1
                    es -= 1
        
            else:
                if self.nblock >= self.nblockmax:
                    raise RuntimeError('BZ_DATA_ERROR: Value nblock higher than maximum')
        
                pp = 0
                nn = self.nextsym - 1
        
                if nn < MTFL_SIZE:
                    pp = self.mtfbase[0]
                    uc = self.mtfa[pp + nn]
                    while nn > 0:
                        self.mtfa[pp + nn] = self.mtfa[pp + nn - 1]
                        nn -= 1
                    self.mtfa[pp] = uc
                else:
                    lno = int(nn / MTFL_SIZE)
                    off = nn % MTFL_SIZE
                    pp = self.mtfbase[lno] + off
                    uc = self.mtfa[pp]
                    
                    while pp > self.mtfbase[lno]:
                        self.mtfa[pp] = self.mtfa[pp - 1]
                        pp -= 1
                    
                    self.mtfbase[lno] += 1
                    
                    while lno > 0:
                        self.mtfbase[lno] -= 1
                        self.mtfa[self.mtfbase[lno]] = self.mtfa[self.mtfbase[lno - 1] + MTFL_SIZE - 1]
                        lno -= 1
                    
                    self.mtfbase[0] -= 1
                    
                    self.mtfa[self.mtfbase[0]] = uc;
                    
                    if self.mtfbase[0] == 0:
                        k = MTFA_SIZE - 1
                        for i in reversed(range(int(256 / MTFL_SIZE))):
                            for j in reversed(range(MTFL_SIZE)):
                                self.mtfa[k] = self.mtfa[self.mtfbase[i] + j]
                                k -= 1
                            self.mtfbase[i] = k + 1
        
                self.unzftab[self.seqtounseq[uc]] += 1
                tt[self.nblock] = self.seqtounseq[uc]
                self.nblock += 1
        
                if self.grouppos == 0:
                    self.groupno += 1
                    if self.groupno >= self.nselectors:
                        raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
                    self.grouppos = BZ_G_SIZE
                    self.gsel = self.selector[self.groupno]
                    self.gminlen = self.minlens[self.gsel]
                    self.glimit = self.limit[self.gsel]
                    self.gperm = self.perm[self.gsel]
                    self.gbase = self.base[self.gsel]
                self.grouppos -= 1
                zn = self.gminlen
        
                zvec = self._get_bits(zn, 'BZ_X_MTF_5')
        
                while True:
                    if zn > 20:
                        raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
                    if zvec <= self.glimit[zn]:
                        break
                    zn += 1
                    zj = self._get_bits(1, 'BZ_X_MTF_6')
                    zvec = (zvec << 1) | zj
        
                if zvec - self.gbase[zn] < 0 or zvec - self.gbase[zn] >= BZ_MAX_ALPHA_SIZE:
                    raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')
                
                self.nextsym = self.gperm[zvec - self.gbase[zn]]

        self.tt = tt

    def _cftable(self):
        """Apply the cumulative frequency table."""
        # These two origin pointer checks need to move to the function where the origin pointer is used. Also, the first check is redundant.
        if self.origptr < 0:
            raise RuntimeError('BZ_DATA_ERROR: Orig ptr is negative')
        if self.origptr > 10 + NSIS_COMPRESS_BZIP2_LEVEL * 100000:
            raise RuntimeError('BZ_DATA_ERROR: Orig ptr is too large')

        cftab = [0] * 257

        for i in range(1, 257):
            cftab[i] = self.unzftab[i - 1] + cftab[i - 1]

        for i in range(self.nblock):
            uc = self.tt[i] & 0xff
            self.tt[cftab[uc]] |= (i << 8)
            cftab[uc] += 1

    def _tpos(self):
        """Calculate the starting position for the inverse Burrows-Wheeler Transform from the origin pointer."""
        nblock_used = 0
        tpos = self.tt[self.origptr] >> 8
        tpos = self.tt[tpos]
        k0 = tpos & 0xff
        tpos >>= 8
        nblock_used += 1

        self.k0 = k0
        self.tpos = tpos
        self.nblock_used = nblock_used

    def _unrle(self):
        """Reverse the run-length encoding."""
        ur = UnRLE(self.avail_out, self.tt, self.nblock, self.nblock_used, self.k0, self.tpos)
        self.out = ur.run()

    def _end_header(self):
        """Check the end header."""
        uc = self._get_bits(8, 'BZ_X_ENDHDR_1')
        if uc != 0x17:
            raise RuntimeError('BZ_DATA_ERROR: Incorrect BZ_X_ENDHDR_1')

    def run(self):
        """Run the decompression process."""
        self._block_header()
        self._origptr()
        self._mapping_table()
        self._selectors()
        self._coding_tables()
        self._huffman_tables()
        self._mtf_values()
        self._mtf_init()
        self._first_mtf()
        self._decompress()
        self._cftable()
        self._tpos()
        self._unrle()
        self._end_header()

In [ ]:
bzd = Bz2Decompress(data, 0x1728)

In [ ]:
bzd.run()

In [ ]:
expected = [6910, 7166, 7422, 7679, 7935, 8191, 8447, 12296, 14859, 15106, 15361, 16992, 17169, 18436, 18943, 18945, 19201, 19458, 19963, 20225, 20481, 20991, 21249, 21505, 21783, 39719, 41216, 41472, 41728, 43008, 43520, 43776, 44544, 46331, 46587, 46843, 47099, 47355, 47611, 47867, 48010, 50767, 51132, 51414, 51713, 51969, 52584, 52784, 52992, 53351, 53555, 53811, 54067, 54323, 54579, 55141, 55411, 55923, 56064, 56320, 56576, 56835, 57194, 57460, 57701, 57970, 58112, 58368, 58866, 59122, 59396, 59652, 59904, 60160, 61184, 61440, 61696, 61952, 62313, 62464, 63488, 64000, 64515, 65024, 65280, 65536, 65869, 66114, 66379, 66919, 67428, 67642, 67947, 68097, 68736, 68868, 69497, 69716, 69989, 70145, 70401, 70660, 71013, 71524, 71726, 72044, 72238, 72506, 72708, 73316, 73586, 73842, 74016, 74349, 74611, 74860, 75070, 75378, 75635, 75808, 76064, 76334, 76576, 76832, 77428, 77569, 77856, 78112, 78407, 78655, 78880, 79690, 80455, 80741, 80930, 81201, 81509, 81675, 81978, 82208, 82464, 82746, 82990, 83570, 83815, 84000, 84325, 84848, 85024, 85364, 85620, 86098, 87317, 88317, 88587, 88832, 89162, 90634, 90900, 91156, 91412, 92672, 92928, 93184, 93559, 94128, 94839, 94986, 95488, 95752, 96256, 96512, 96966, 97030, 97280, 98167, 99322, 99561, 99828, 99959, 100864, 101120, 101376, 101632, 101888, 102144, 102400, 102656, 103486, 103781, 104043, 104305, 104522, 104778, 104961, 105219, 105477, 105730, 105984, 106240, 106496, 106774, 107008, 107264, 107522, 107776, 108032, 108800, 109056, 109312, 109568, 109824, 110080, 110336, 110644, 110848, 111104, 111392, 111616, 111872, 112128, 112384, 112640, 112896, 113408, 113664, 113920, 115200, 115456, 115712, 115968, 116225, 116480, 116736, 116992, 117248, 117904, 118160, 118416, 118528, 118784, 119040, 119296, 119552, 121344, 121610, 123139, 123395, 123648, 123952, 124416, 124930, 125184, 125444, 125696, 125952, 126208, 134144, 135680, 135936, 160773, 161280, 161540, 162304, 162560, 162816, 164096, 164354, 165632, 165888, 170501, 171008, 171264, 171776, 172032, 172288, 172544, 173056, 173317, 177664, 177920, 178176, 178432, 179200, 179712, 180225, 180736, 180992, 181760, 182016, 182272, 182528, 183040, 183296, 184320, 184832, 185088, 185600, 186112, 186368, 186624, 187008, 187904, 188160, 188672, 188928, 189184, 189440, 189696, 189954, 190213, 194304, 194561, 194821, 195072, 196352, 197376, 197632, 197888, 198144, 198656, 198912, 199424, 200960, 201216, 201600, 201984, 202240, 202496, 202752, 203136, 203264, 203520, 204544, 206336, 207361, 209408, 225538, 239874, 276738, 276997, 300544, 300802, 326401, 331008, 439811, 451840, 454656, 454912, 455170, 455426, 455682, 455938, 456194, 457472, 457728, 457984, 458240, 458497, 458755, 460545, 461316, 461568, 461824, 462080, 462336, 462596, 463360, 463620, 463873, 464128, 464384, 464643, 464896, 465152, 465409, 465664, 465920, 466176, 466432, 466688, 466948, 467203, 467456, 467716, 467970, 468227, 468480, 468736, 468992, 469248, 469506, 469763, 470271, 470272, 470528, 470784, 471040, 471296, 471552, 471808, 472064, 472321, 472578, 472832, 473088, 473344, 473600, 473856, 474112, 474368, 474624, 474880, 475136, 476416, 477184, 477440, 477696, 477952, 478208, 478464, 478720, 481536, 481824, 482080, 483072, 483840, 486144, 486400, 486656, 486912, 487168, 488192, 488448, 488704, 488960, 489216, 489472, 489728, 489984, 490496, 490752, 493066, 493312, 493568, 493824, 494172, 494370, 494624, 494880, 495104, 495360, 495872, 496128, 496384, 496640, 496896, 498944, 499456, 499967, 501504, 501760, 502016, 502528, 502784, 503040, 503296, 505088, 506125, 506381, 506637, 506893, 507149, 507405, 507648, 508416, 508682, 508960, 509450, 509810, 509996, 510208, 510720, 510976, 511232, 511490, 511744, 512255, 512256, 512512, 513280, 514048, 514560, 516096, 517178, 517434, 517690, 517946, 518202, 518458, 518714, 518970, 519269, 519482, 519738, 519994, 520556, 520812, 521006, 521262, 521530, 521788, 522103, 522356, 522603, 522859, 523119, 523322, 523553, 523897, 524141, 524398, 524654, 524910, 525056, 525428, 525675, 525882, 526138, 526394, 526592, 526848, 527211, 527470, 527980, 528211, 528499, 528686, 529006, 529273, 529509, 529772, 530036, 530277, 530476, 530804, 531045, 531301, 531564, 531823, 532082, 532327, 532594, 532833, 533109, 533306, 533615, 533863, 534131, 534373, 534629, 534900, 535154, 535399, 535668, 535909, 536165, 536421, 536684, 537452, 537711, 537967, 538223, 538479, 538735, 538982, 539247, 539493, 539749, 540004, 540261, 540517, 540786, 541029, 541228, 541550, 541797, 542052, 542252, 542575, 542831, 543087, 543339, 543595, 543860, 544110, 544368, 544628, 544884, 545140, 545392, 545652, 545895, 546162, 546420, 546676, 546931, 547176, 547429, 547692, 547956, 548217, 548463, 548709, 548964, 549220, 549485, 549749, 550004, 550256, 550517, 550772, 551026, 551282, 551539, 551795, 552051, 552545, 552704, 553061, 553216, 553728, 553986, 554528, 554752, 555008, 555264, 555552, 555842, 556398, 556654, 556800, 557056, 557422, 557682, 557934, 558194, 558336, 558592, 558949, 559214, 559461, 559662, 559986, 560236, 560498, 560686, 560997, 561203, 561516, 561713, 561970, 562284, 562483, 562796, 563060, 563456, 563761, 564224, 564480, 564782, 564992, 565760, 566016, 566272, 566646, 566784, 567296, 567662, 567916, 568172, 568428, 568676, 568932, 569189, 569458, 569701, 569972, 570215, 570484, 570738, 570994, 571237, 571495, 571648, 571904, 572160, 572416, 572788, 572995, 573440, 573728, 573952, 574322, 574464, 574752, 574976, 575232, 575526, 575776, 576000, 576256, 576512, 576768, 577034, 577280, 577536, 577824, 578086, 578396, 578560, 578848, 579072, 579328, 579616, 579840, 580188, 580352, 580608, 580864, 581120, 581408, 581747, 582003, 582144, 582400, 582740, 582912, 583168, 583424, 583680, 583936, 584192, 584448, 584736, 584992, 585326, 585581, 585760, 586016, 586272, 586835, 587018, 587302, 587552, 587808, 588064, 588320, 588576, 588800, 589056, 589312, 589824, 590112, 590624, 590940, 591215, 591360, 591737, 591910, 592160, 592384, 592640, 592896, 593152, 593440, 593664, 593920, 594268, 594944, 595311, 595466, 595785, 596046, 596293, 596480, 596736, 596992, 597280, 597504, 597760, 598016, 598272, 598528, 598784, 599040, 599404, 599584, 599891, 600064, 600412, 600668, 600924, 601120, 601376, 601632, 601856, 602204, 602484, 602716, 602880, 603763, 604005, 604218, 604780, 605029, 605294, 605556, 606976, 607264, 607604, 607852, 608112, 608368, 608578, 608882, 609138, 609312, 609654, 609908, 610164, 610407, 610672, 610926, 611188, 611444, 611700, 611956, 612212, 612468, 612724, 612980, 613236, 613492, 613748, 614004, 614260, 614516, 614772, 615028, 615284, 615540, 615796, 616052, 616308, 616564, 616820, 617074, 617330, 617550, 617795, 618051, 618272, 618528, 618784, 619127, 619383, 619631, 619895, 620151, 620404, 620644, 620901, 621157, 621422, 621676, 621932, 622188, 622444, 622624, 622956, 623104, 623457, 623728, 623973, 624193, 624481, 624737, 625006, 625262, 625505, 625769, 626025, 626281, 626537, 626789, 627045, 627232, 627488, 627744, 628000, 628256, 628581, 628768, 629024, 629353, 629605, 629861, 630117, 630373, 630629, 630881, 631137, 631397, 631653, 631922, 632165, 632430, 632686, 632933, 633189, 633445, 633632, 633888, 634220, 634476, 634732, 634988, 635244, 635500, 635756, 636012, 636198, 636448, 636783, 636960, 637294, 637440, 637811, 638072, 638328, 638584, 638829, 639096, 639352, 639852, 640115, 640355, 640616, 640872, 641140, 641384, 641654, 641896, 642156, 642408, 642664, 642928, 643188, 643432, 643683, 643954, 644200, 644472, 644725, 644972, 645235, 645492, 645740, 645996, 646258, 646514, 646770, 647026, 647282, 647528, 647784, 648036, 648300, 648556, 648812, 649068, 649324, 649588, 649844, 650100, 650356, 650610, 650864, 651107, 651379, 651603, 651859, 652147, 652371, 652612, 652868, 653172, 653394, 653680, 653938, 654194, 654446, 654702, 654958, 655214, 655442, 655730, 655972, 656228, 656488, 656740, 656996, 657252, 657508, 657772, 658020, 658276, 658540, 658790, 658976, 659286, 659564, 659820, 660076, 660332, 660594, 660820, 661076, 661332, 661588, 661828, 662116, 662340, 662636, 662892, 663148, 663404, 663634, 663891, 664147, 664366, 664622, 664878, 665134, 665390, 665646, 665934, 666190, 666479, 666726, 666985, 667168, 667424, 667680, 667936, 668192, 668448, 668704, 668960, 669216, 669472, 669728, 669984, 670319, 670575, 670831, 671087, 671337, 671854, 672110, 672366, 672622, 672878, 673134, 673390, 673646, 673889, 674121, 674415, 674671, 674816, 675072, 675443, 675700, 675956, 676212, 676468, 676724, 676980, 677236, 677492, 677731, 677955, 678260, 678516, 678739, 679022, 679276, 679532, 679788, 680044, 680269, 680548, 680804, 681057, 681318, 681574, 681830, 682086, 682310, 682566, 682822, 683078, 683383, 683617, 683873, 684064, 684320, 684641, 684832, 685172, 685399, 685682, 685932, 686195, 686446, 686711, 686964, 687220, 687476, 687662, 687991, 688238, 688416, 688672, 688928, 689184, 689440, 689774, 690030, 690286, 690464, 690720, 690976, 691310, 691566, 691744, 692000, 692334, 692512, 692852, 693108, 693364, 693620, 693876, 694132, 694387, 694644, 694900, 695147, 695404, 695659, 695905, 696132, 696388, 696693, 696936, 697207, 697412, 697710, 697970, 698226, 698482, 698624, 698880, 699235, 699747, 700003, 700259, 700515, 700771, 701027, 701299, 701523, 701797, 702060, 702316, 702572, 702828, 703084, 703329, 703596, 703852, 704108, 704364, 704620, 704876, 705132, 705388, 705644, 705900, 706156, 706412, 706665, 706924, 707180, 707436, 707692, 707952, 708207, 708463, 708719, 708975, 709231, 709487, 709743, 709999, 710249, 710505, 710761, 711017, 711266, 711525, 711781, 712037, 712293, 712549, 712812, 713068, 713321, 713577, 713833, 714089, 714341, 714597, 714864, 715120, 715331, 715619, 715843, 716099, 716396, 716611, 716897, 717153, 717409, 717665, 717921, 718177, 718441, 718689, 718945, 719201, 719457, 719713, 719969, 720225, 720481, 720737, 720993, 721249, 721505, 721761, 722017, 722273, 722529, 722785, 723055, 723317, 723523, 723820, 724073, 724329, 724588, 724736, 725103, 725345, 725605, 725857, 726113, 726383, 726639, 726893, 727149, 727397, 727663, 727919, 728175, 728431, 728687, 728943, 729199, 729344, 729711, 729967, 730223, 730479, 730729, 730991, 731241, 731503, 731745, 732015, 732265, 732527, 732777, 733039, 733295, 733551, 733801, 734049, 734325, 734561, 734817, 735081, 735343, 735599, 735855, 736111, 736361, 736617, 736873, 737129, 737385, 737641, 737897, 738153, 738409, 738661, 738921, 739189, 739445, 739701, 739925, 740181, 740469, 740713, 740967, 741217, 741487, 741705, 741993, 742249, 742473, 742761, 743017, 743241, 743529, 743785, 744041, 744297, 744553, 744809, 745065, 745321, 745577, 745801, 746057, 746345, 746601, 746857, 747113, 747337, 747621, 747873, 748129, 748389, 748655, 748901, 749157, 749413, 749669, 749929, 750164, 750452, 750708, 750964, 751220, 751444, 751732, 751988, 752244, 752500, 752756, 753012, 753268, 753486, 753742, 754018, 754253, 754464, 754803, 755059, 755283, 755539, 755826, 756082, 756294, 756550, 756838, 757094, 757350, 757606, 757862, 758118, 758374, 758644, 758851, 759107, 759363, 759619, 759907, 760163, 760419, 760675, 760931, 761197, 761449, 761705, 761961, 762144, 762468, 762729, 762985, 763245, 763497, 763753, 764016, 764272, 764528, 764784, 765033, 765283, 765538, 765812, 765984, 766240, 766531, 766752, 767090, 767346, 767602, 767858, 768102, 768366, 768579, 768866, 769132, 769394, 769647, 769902, 770169, 770425, 770681, 770925, 771176, 771436, 771684, 771954, 772210, 772469, 772719, 772969, 773237, 773459, 773715, 773989, 774249, 774521, 774768, 775023, 775283, 775533, 775789, 776045, 776301, 776557, 776813, 777065, 777325, 777589, 777839, 778063, 778356, 778607, 779109, 779369, 779621, 779881, 780137, 780389, 780655, 780911, 781167, 781413, 781679, 781925, 782191, 782447, 782693, 782927, 783205, 783461, 783717, 783973, 784229, 784485, 784756, 785012, 785255, 785511, 785761, 786031, 786273, 786529, 786755, 787043, 787232, 787561, 787813, 788082, 788256, 788592, 788852, 789111, 789367, 789623, 789840, 790096, 790386, 790642, 790898, 791154, 791395, 791618, 791846, 792181, 792389, 792645, 792901, 793189, 793455, 793701, 793953, 794223, 794482, 794740, 794996, 795237, 795493, 795758, 796012, 796268, 796521, 796788, 797044, 797300, 797556, 797797, 798053, 798327, 798575, 798839, 799095, 799264, 799520, 799849, 800115, 800370, 800544, 800876, 801135, 801385, 801637, 801893, 802149, 802405, 802661, 802926, 803182, 803438, 803694, 803950, 804206, 804462, 804718, 804974, 805230, 805486, 805742, 805998, 806254, 806510, 806766, 807022, 807278, 807534, 807790, 808046, 808302, 808558, 808736, 809081, 809317, 809589, 809760, 810099, 810355, 810611, 810867, 811128, 811366, 811619, 811875, 812142, 812405, 812643, 812914, 813155, 813411, 813688, 813934, 814190, 814450, 814631, 814887, 815209, 815471, 815715, 815971, 816238, 816486, 816737, 816997, 817253, 817523, 817779, 818035, 818291, 818547, 818803, 819059, 819315, 819571, 819827, 820083, 820339, 820595, 820851, 821107, 821363, 821619, 821875, 822131, 822387, 822643, 822899, 823155, 823411, 823653, 823909, 824169, 824421, 824677, 824944, 825185, 825459, 825632, 825888, 826144, 826400, 826656, 826912, 827168, 827424, 827759, 827936, 828275, 828521, 828769, 829033, 829294, 829537, 829793, 830049, 830305, 830561, 830832, 831088, 831264, 831520, 831776, 832032, 832288, 832544, 832800, 833056, 833312, 833568, 833824, 834163, 834419, 834677, 834936, 835192, 835411, 835616, 835941, 836206, 836462, 836718, 836974, 837221, 837477, 837734, 837990, 838176, 838400, 838767, 839023, 839279, 839534, 839793, 840014, 840224, 840540, 840796, 841052, 841248, 841588, 841844, 842098, 842307, 842563, 842864, 843087, 843296, 843617, 843887, 845167, 845344, 845600, 845940, 846196, 846368, 846703, 846940, 847136, 847392, 847648, 847904, 848239, 848495, 848751, 848997, 849253, 849509, 849765, 850021, 850277, 850533, 850789, 851013, 851269, 851456, 851814, 852080, 852338, 852594, 852833, 853024, 853280, 853536, 853876, 854099, 854528, 854784, 856832, 858112, 858368, 858624, 858880, 859139, 859394, 859650, 859905, 860160, 860416, 860672, 860928, 861184, 861440, 861698, 861955, 862211, 862467, 862723, 862979, 863232, 863491, 863747, 864000, 864256, 864512, 864768, 865024, 866304, 866561, 866817, 867072, 867328, 867584, 869120, 869376, 869888, 870144, 870400, 872192, 872960, 873984, 874496, 875264, 875776, 878336, 878592, 878848, 879104, 879360, 879616, 880128, 880384, 880640, 880896, 881152, 881408, 881664, 881920, 882176, 882432, 882688, 882944, 883200, 883456, 883712, 883968, 884224, 884480, 884736, 884992, 885248, 917504, 917760, 918272, 918528, 918784, 919040, 2718, 3854, 4347, 5058, 5120, 5740, 5894, 11264, 11520, 23808, 25344, 25600, 32000, 49664, 59180, 72994, 79936, 85833, 87899, 91770, 92304, 94641, 96014, 103153, 457216, 462848, 463104, 877787, 878043, 878299, 887040, 2373, 4395, 50432, 52268, 64507, 68379, 79163, 86367, 86625, 86884, 87655, 89344, 89600, 89856, 90112, 90368, 98666, 100333, 103412, 124160, 161792, 456704, 456960, 459008, 563200, 606720, 865280, 15619, 20992, 49920, 62992, 63478, 88338, 92020, 95376, 97992, 99046, 100590, 456448, 459264, 459520, 459776, 460032, 460288, 460800, 461056, 490240, 505344, 3337, 17920, 18176, 24320, 25856, 27648, 65003, 67086, 92452, 93856, 94387, 97726, 98530, 553472, 590591, 917248, 50176, 66580, 69153, 71215, 79417, 80197, 87116, 639488, 699392, 778752, 44296, 476928, 515072, 573184, 508160, 2047, 43272, 502527, 504063, 505855, 510719, 495616, 40195, 42760, 62788, 113152, 121856, 122368, 182784, 192512, 199936, 879872, 2168, 35079, 39424, 119808, 120064, 120448, 120576, 120832, 121088, 475648, 481032, 497408, 565248, 491008, 514816, 589568, 3072, 491520, 40448, 40959, 41215, 499968, 39167, 51502, 854272, 6184, 594432, 604432, 484614, 876996, 877252, 877508, 918016, 865708, 865964, 866220, 28672, 30464, 30720, 31232, 31488, 32256, 32512, 33280, 35584, 35840, 37120, 37888, 55681, 108288, 108544, 114688, 114944, 122112, 162048, 163072, 179456, 180480, 181504, 183552, 184576, 185344, 187136, 190464, 190720, 191488, 191744, 192000, 193024, 193280, 193536, 193792, 194048, 195328, 195584, 197120, 198400, 201856, 204032, 205568, 205824, 206080, 209664, 211712, 220672, 220928, 221184, 225024, 229632, 229888, 230144, 230400, 230656, 231168, 231424, 236032, 236288, 238848, 239360, 261120, 269312, 269568, 269824, 270080, 270336, 270592, 270848, 271104, 271360, 271616, 271872, 272128, 285952, 286336, 286848, 290304, 290560, 290816, 291072, 291328, 292352, 292608, 292864, 293632, 293888, 294400, 356096, 363776, 367616, 367872, 368384, 388608, 389632, 400640, 400896, 401920, 410112, 411136, 427008, 427264, 427520, 427776, 428032, 428288, 428544, 428800, 429312, 432640, 432896, 433152, 433408, 433664, 433920, 434176, 434432, 434688, 434944, 435200, 436992, 439552, 441600, 442624, 444672, 445696, 445952, 446720, 447488, 447744, 448000, 448256, 453376, 453632, 453888, 603521, 132608, 500224, 34304, 114432, 479236, 497664, 554240, 606500, 181248, 183808, 192768, 196864, 238592, 393216, 6655, 416000, 416256, 514304, 483584, 122880, 139264, 149504, 150528, 478976, 484096, 26624, 27136, 30976, 36352, 130048, 130304, 137472, 166912, 167936, 171520, 266496, 266752, 267008, 267264, 267520, 267776, 289536, 513794, 500480, 12032, 63853, 34560, 168960, 170752, 169216, 12815, 13311, 13312, 13568, 13824, 168448, 169728, 54929, 500736, 23296, 27392, 35328, 36096, 126464, 126720, 126976, 127232, 127488, 127744, 128000, 128256, 128768, 129024, 129280, 130560, 132352, 134912, 135168, 135424, 142080, 208128, 484870, 130816, 29696, 48128, 33024, 479488, 226560, 22272, 163328, 210944, 392960, 178944, 220160, 220416, 279808, 316928, 317440, 317696, 318208, 325120, 360448, 360704, 360960, 361216, 368128, 370176, 388096, 443648, 443904, 254720, 254976, 263936, 264448, 297984, 298240, 299264, 505856, 200704, 393728, 393984, 394240, 451328, 451584, 483328, 500992, 284160, 284416, 284672, 284928, 357888, 358144, 32768, 34047, 200192, 275968, 343296, 344064, 344832, 347392, 347648, 348928, 479744, 33536, 39936, 49168, 49408, 22528, 501252, 22016, 282112, 355840, 200448, 219904, 268032, 268288, 355072, 355328, 441344, 10496, 379648, 384256, 444416, 391168, 391449, 38784, 255488, 257536, 265728, 136960, 192256, 204288, 253696, 253952, 254464, 265984, 266240, 280576, 303104, 356864, 357120, 375040, 375296, 436736, 454400, 24832, 188416, 262912, 263168, 263424, 263680, 351744, 353024, 341504, 341760, 261376, 287232, 480000, 114304, 184096, 185856, 195840, 199168, 204825, 205056, 205312, 206592, 207104, 441984, 442240, 442496, 447268, 485122, 2816, 141568, 160512, 225792, 226816, 227072, 227840, 233216, 233472, 275712, 282880, 285440, 285696, 286464, 297728, 304896, 318464, 318720, 318976, 319232, 319488, 319744, 320256, 320512, 320768, 321024, 321280, 321536, 321792, 322048, 322304, 322560, 322816, 323072, 323328, 323584, 323840, 324096, 324352, 326912, 327424, 327680, 333312, 335616, 336128, 336384, 342784, 349440, 349696, 387072, 387584, 387840, 395264, 418048, 426496, 430080, 430848, 431104, 431360, 431616, 431872, 444928, 453120, 485376, 312832, 355584, 367104, 370432, 242432, 245760, 253184, 279552, 301056, 301312, 301568, 301824, 302080, 302336, 302592, 317184, 361472, 361728, 361984, 362240, 362496, 366848, 388352, 392704, 412928, 413184, 413952, 414464, 414720, 417024, 417280, 23040, 26368, 27904, 148224, 150272, 157184, 157440, 174592, 174848, 222976, 250112, 258048, 258304, 258816, 259072, 259328, 259584, 260096, 260352, 264192, 282368, 282624, 364032, 373248, 485632, 14080, 16384, 25088, 26112, 34048, 34816, 37376, 48389, 128512, 138240, 139008, 139776, 140032, 143104, 143360, 144640, 144896, 145152, 147712, 147968, 148480, 148736, 149248, 150016, 155648, 156928, 161024, 166144, 166656, 168192, 175104, 175616, 177152, 207872, 208640, 223232, 223488, 226304, 229120, 229376, 230912, 231936, 232192, 232448, 232704, 232960, 233728, 233984, 234496, 235264, 235520, 235776, 303360, 313088, 313344, 313600, 313856, 314112, 315904, 316160, 327168, 328960, 340224, 349184, 349952, 350464, 350720, 350976, 351232, 375552, 380416, 380928, 381696, 382720, 383232, 384000, 384512, 384768, 385024, 385280, 385536, 385792, 389120, 394496, 395008, 396544, 396800, 399104, 399360, 402944, 403200, 403456, 403712, 403968, 410624, 418304, 418560, 424960, 425216, 425728, 425984, 437248, 438528, 438784, 449280, 449536, 449792, 450048, 450304, 450560, 450816, 451072, 147200, 260864, 268800, 283136, 283392, 283648, 283904, 358400, 358656, 358912, 359168, 359424, 359680, 359936, 369664, 412672, 417792, 439040, 439296, 452096, 12799, 22784, 36864, 141056, 142592, 144128, 154112, 176128, 177408, 212736, 342528, 386560, 386816, 485888, 11776, 155392, 242688, 242944, 243456, 243968, 244480, 244736, 245504, 246272, 249600, 249856, 258560, 298752, 372736, 19968, 228096, 228352, 228608, 228864, 231680, 269056, 273408, 308224, 311808, 312064, 312320, 312576, 314880, 315136, 315392, 315648, 320000, 325632, 325888, 332288, 332800, 333824, 334336, 335360, 336640, 337920, 338176, 338432, 338688, 338944, 339200, 339456, 339712, 339968, 340480, 342272, 343552, 343808, 344320, 344576, 345088, 345344, 345600, 345856, 346112, 346368, 346624, 346880, 347136, 347904, 348160, 348416, 348672, 351488, 363008, 363264, 363520, 364288, 364544, 365056, 365312, 366592, 374528, 375808, 378624, 380672, 381184, 381440, 388864, 397824, 401152, 402688, 416512, 425472, 429824, 430336, 15872, 487424, 23552, 48645, 131584, 131840, 134656, 136192, 151552, 151808, 296960, 297472, 26880, 29440, 129536, 129792, 136704, 138496, 140288, 145408, 145664, 155904, 167424, 168704, 169472, 169984, 173824, 174080, 174336, 203776, 208384, 210176, 224000, 224256, 224512, 224768, 225280, 236544, 236800, 237056, 237312, 237568, 237824, 238080, 238336, 241920, 244224, 247040, 247808, 248064, 250368, 250624, 250880, 251136, 251392, 259840, 260608, 261632, 261888, 262144, 262400, 264704, 264960, 265216, 265472, 281088, 281344, 281600, 281856, 287744, 297216, 303616, 303872, 304128, 304384, 304640, 305152, 305408, 305664, 305920, 306176, 306432, 306688, 306944, 307200, 307456, 307712, 307968, 308480, 308736, 308992, 309248, 314368, 314624, 317952, 325376, 326144, 370688, 372992, 397312, 397568, 402176, 476672, 28928, 133120, 157696, 191232, 241152, 328448, 328704, 362752, 364800, 372480, 377088, 377344, 377600, 377856, 378112, 378368, 378880, 563968, 133376, 133632, 133888, 136448, 137728, 149760, 152320, 163584, 163840, 164608, 165120, 166400, 173568, 190976, 208896, 213248, 223744, 227328, 227584, 234752, 235008, 239616, 256512, 256768, 257024, 257280, 273664, 273920, 274176, 274432, 274688, 274944, 275200, 275456, 280832, 288256, 290048, 291584, 291840, 292096, 293120, 293376, 294144, 299520, 369920, 371456, 397056, 404224, 404480, 404736, 404992, 405248, 405504, 405760, 406016, 406272, 406528, 406784, 407040, 407296, 407552, 407808, 408064, 408320, 408576, 408832, 409088, 409344, 409600, 409856, 413440, 415232, 415488, 417536, 430592, 437504, 437760, 438016, 438272, 440832, 132096, 140544, 142336, 145920, 146176, 146432, 146688, 146944, 147456, 150784, 151040, 151296, 156672, 196096, 199680, 221952, 239104, 268544, 272384, 272640, 272896, 273152, 276224, 276480, 309760, 310016, 310272, 310528, 310784, 311040, 311296, 311552, 324608, 326656, 327936, 328192, 329216, 329472, 329728, 329984, 330240, 330496, 330752, 331264, 331520, 331776, 332032, 332544, 333056, 333568, 334080, 334592, 334848, 335104, 336896, 337152, 337408, 337664, 343040, 350208, 371200, 374272, 376576, 379392, 380160, 381952, 382208, 382464, 382976, 383488, 383744, 387328, 394752, 395520, 399872, 402432, 416768, 429056, 440064, 440320, 440576, 445184, 445440, 446976, 448512, 448768, 449024, 37632, 152576, 153600, 158464, 210432, 210688, 212992, 226048, 244992, 252928, 255744, 309504, 316416, 316672, 365568, 365824, 366080, 366336, 376320, 389888, 426240, 432128, 432384, 444160, 452352, 48901, 441088, 16640, 28160, 28416, 29952, 36608, 122624, 140800, 141312, 143872, 148992, 154368, 159232, 159488, 164864, 165376, 167168, 167680, 175360, 176640, 176896, 179968, 211200, 211456, 219392, 219648, 234240, 246016, 248320, 248576, 248832, 249088, 249344, 252672, 256000, 256256, 257792, 262656, 287488, 299776, 300032, 300288, 357376, 357632, 368640, 368896, 369152, 369408, 370944, 373504, 373760, 389376, 391680, 391936, 392192, 392448, 395776, 401664, 414208, 415744, 443392, 452608, 452864, 14336, 14592, 29184, 30208, 137216, 142848, 155136, 159744, 160000, 160256, 187392, 187648, 207616, 240128, 242176, 247296, 253440, 254208, 277248, 288000, 296192, 302848, 356352, 356608, 376832, 401408, 411392, 411648, 411904, 412160, 418816, 419072, 419328, 419584, 419840, 420096, 420352, 420608, 420864, 421120, 421376, 421632, 421888, 422144, 422400, 422656, 422912, 423168, 423424, 423680, 423936, 424192, 424448, 424704, 426752, 429568, 435456, 435712, 16128, 31744, 38144, 38400, 131328, 134400, 138752, 139520, 143616, 144384, 152064, 152832, 153088, 153344, 153856, 154624, 154880, 156160, 158208, 158976, 170240, 175872, 176384, 178688, 206848, 209152, 209920, 212224, 212480, 213504, 213760, 214016, 214272, 214528, 214784, 215040, 215296, 215552, 215808, 216064, 216320, 216576, 216832, 217088, 217344, 217600, 217856, 218112, 218368, 218624, 218880, 219136, 222720, 243200, 245248, 247552, 251648, 251904, 252160, 252416, 255232, 277504, 277760, 278016, 278272, 278528, 278784, 279040, 279296, 280064, 280320, 286976, 288768, 289024, 289280, 294656, 294912, 295168, 295424, 295680, 295936, 296448, 296704, 352000, 352256, 352512, 352768, 353280, 353536, 353792, 354048, 354304, 354560, 354816, 360192, 367360, 379904, 386048, 386304, 390144, 396032, 396288, 398080, 398336, 398592, 398848, 442880, 443136, 446208, 446464, 454144, 491776, 141824, 157952, 158720, 246784, 298496, 324864, 335872, 340736, 340992, 341248, 342016, 374016, 374784, 379136, 393472, 410880, 413696, 435968, 172800, 211968, 243712, 42112, 42624, 44928, 45952, 131072, 221440, 221696, 222208, 222464, 285184, 288512, 289792, 299008, 390400, 390656, 390912, 399616, 400128, 400384, 240384, 240640, 240896, 241408, 241664, 246528, 412416, 414976, 436224, 436480, 507904, 24576, 132864, 137984, 156416, 196608, 371712, 371968, 372224, 376064, 410368, 480278, 876544, 24064, 77214, 83358, 84629, 509312, 527746, 537019, 537275, 594688, 603285, 605845, 606101, 606357, 844032, 844288, 844544, 844800, 855808, 520192, 552192, 856064, 10240, 60416, 60672, 60928, 480512, 492032, 567040, 855552, 857088, 857344, 857600, 857856, 475392, 855040, 855296, 497920, 516352, 516608, 516864, 42240, 480770, 867871, 868127, 868383, 498176, 856320, 856576, 10752, 498432, 476160, 515505, 515761, 516017, 907775, 914687, 44032, 914944, 908031, 492288, 912640, 915200, 916991, 913151, 911103, 913663, 913919, 914175, 915456, 916735, 917247, 914688, 912383, 912639, 913407, 911872, 907519, 11008, 915967, 916223, 907008, 907007, 908032, 914431, 916479, 911740, 482333, 482589, 482845, 910337, 910593, 911361, 498688, 492544, 45312, 497152, 487680, 492800, 481290, 919296, 17408, 17664, 45568, 487936, 491264, 45056, 4608, 8448, 8704, 8960, 9216, 9472, 9728, 9984, 475904, 484352, 39168, 256, 512, 1023, 1279, 1535, 1791, 3839, 5631, 100609, 117759, 124927, 499455, 503807, 504319, 504575, 504831, 505087, 513023, 513279, 513791, 556287, 565759, 586751, 671743, 868863, 869119, 869887, 870911, 871167, 871423, 871679, 871935, 872191, 872703, 872959, 873471, 873727, 873983, 874495, 875007, 875263, 875775, 876287, 876543, 885759, 886015, 886271, 886527, 886783, 887039, 887551, 887807, 888063, 888319, 888575, 888831, 889087, 889343, 889599, 889855, 890111, 890367, 890623, 890879, 891135, 891391, 891647, 891903, 892159, 892415, 892671, 892927, 893183, 893439, 893695, 893951, 894207, 894463, 894719, 894975, 895231, 895487, 895743, 895999, 896255, 896473, 896728, 896981, 897221, 897479, 897753, 898047, 898303, 898559, 898815, 899071, 899327, 899583, 899839, 900059, 900315, 900556, 900863, 901083, 901338, 901588, 901843, 902099, 902344, 902603, 902867, 903117, 903374, 903631, 903897, 904133, 904402, 904646, 904905, 905168, 905430, 905687, 905945, 906193, 906442, 906705, 908292, 908544, 908800, 909086, 909312, 909568, 909824, 910080, 911345]

In [ ]:
tt = [z for z in bzd.tt if z]

In [ ]:
self.assertListEqual(tt, expected)

In [ ]:
sha256 = hashlib.sha256(bzd.out).hexdigest()

In [ ]:
self.assertEqual(sha256, 'eeaefe8c8a5d42855d886e5368d5c752a03d821681c7b2ceb5d900aa6cf70e18')

In [ ]:
bzd = Bz2Decompress(data, 0x1728)

In [ ]:
uc = bzd._get_bits(8, 'BZ_X_BLKHDR_1')

In [ ]:
hex(uc)

In [ ]:
self.assertEqual(uc, 0x31)

In [ ]:
if uc != 0x31:
    raise RuntimeError('BZ_DATA_ERROR: Incorrect BZ_X_BLKHDR_1')

In [ ]:
bzd.origptr = 0

In [ ]:
uc = bzd._get_bits(8, 'BZ_X_ORIGPTR_1')

In [ ]:
hex(uc)

In [ ]:
self.assertEqual(uc, 0x0)

In [ ]:
bzd.origptr = (bzd.origptr << 8) | uc

In [ ]:
hex(bzd.origptr)

In [ ]:
self.assertEqual(bzd.origptr, 0x0)

In [ ]:
uc = bzd._get_bits(8, 'BZ_X_ORIGPTR_2')

In [ ]:
hex(uc)

In [ ]:
self.assertEqual(uc, 0xd)

In [ ]:
bzd.origptr = (bzd.origptr << 8) | uc

In [ ]:
hex(bzd.origptr)

In [ ]:
self.assertEqual(bzd.origptr, 0xd)

In [ ]:
uc = bzd._get_bits(8, 'BZ_X_ORIGPTR_3')

In [ ]:
hex(uc)

In [ ]:
self.assertEqual(uc, 0xb)

In [ ]:
bzd.origptr = (bzd.origptr << 8) | uc

In [ ]:
hex(bzd.origptr)

In [ ]:
self.assertEqual(bzd.origptr, 0xd0b)

In [ ]:
if bzd.origptr < 0:
    raise RuntimeError('BZ_DATA_ERROR: Orig ptr is negative')

In [ ]:
if bzd.origptr > 10 + NSIS_COMPRESS_BZIP2_LEVEL * 100000:
    raise RuntimeError('BZ_DATA_ERROR: Orig ptr is too large')

In [ ]:
# Mapping table

In [ ]:
inuse16 = list()
for i in range(16):
    uc = bzd._get_bits(1, 'BZ_X_MAPPING_1')
    if uc == 1:
        inuse16.append(True)
    else:
        inuse16.append(False)
bzd.inuse16 = inuse16

In [ ]:
self.assertListEqual(bzd.inuse16, [True] * 16)

In [ ]:
bzd.inuse = [False] * 256

In [ ]:
len(bzd.inuse)

In [ ]:
for i in range(16):
    if bzd.inuse16[i]:
        for j in range(16):
            uc = bzd._get_bits(1, 'BZ_X_MAPPING_2')
            if uc == 1:
                bzd.inuse[i * 16 + j] = True

In [ ]:
expected = [True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, True, True, False, True, False, True, False, True, True, True, True, True, True, False, True, False, True, True, True, False, False, True, True, False, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, False, False, False, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, False, False, False, True, True, True, False, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, True, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, True, False, True, False, False, False, False, False, False, False, True, True, False, True, False, False, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, True, False, False, False, True, False, False, True, False, True, False, True, True, False, False, True, True, False, True, False, True, False, False, False, True, True, False, True, True, True]

In [ ]:
self.assertListEqual(bzd.inuse, expected)

In [ ]:
bzd.ninuse = 0

In [ ]:
bzd.seqtounseq = [0] * 256

In [ ]:
for qi in range(256):
    if bzd.inuse[qi]:
        bzd.seqtounseq[bzd.ninuse] = qi
        bzd.ninuse += 1

In [ ]:
self.assertEqual(bzd.ninuse, 158)

In [ ]:
expected = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 36, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 128, 129, 130, 138, 144, 145, 149, 158, 160, 172, 176, 177, 179, 187, 188, 190, 194, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 226, 230, 233, 235, 237, 238, 241, 242, 244, 246, 250, 251, 253, 254, 255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(bzd.seqtounseq, expected)

In [ ]:
if bzd.ninuse == 0:
    raise RuntimeError('BZ_DATA_ERROR: ninuse is zero')

In [ ]:
alphasize = bzd.ninuse + 2

In [ ]:
alphasize

In [ ]:
self.assertEqual(alphasize, 160)

In [ ]:
# Selectors

In [ ]:
ngroups = bzd._get_bits(3, 'BZ_X_SELECTOR_1')

In [ ]:
ngroups

In [ ]:
self.assertEqual(ngroups, 5)

In [ ]:
if ngroups < 2 or ngroups > 6:
    raise RuntimeError('BZ_DATA_ERROR: Invalid number of groups')

In [ ]:
nselectors = bzd._get_bits(15, 'BZ_X_SELECTOR_2')

In [ ]:
nselectors

In [ ]:
self.assertEqual(nselectors, 37)

In [ ]:
if nselectors < 1:  # Maybe this can be nselectors == 0 ?
    raise RuntimeError('BZ_DATA_ERROR: Invalid number of selectors')

In [ ]:
bzd.selectormtf = [0] * BZ_MAX_SELECTORS

In [ ]:
for i in range(nselectors):
    j = 0
    while True:
        uc = bzd._get_bits(1, 'BZ_X_SELECTOR_3')
        if uc == 0:  # Why not store this value as 3 bits rather than a set of bits? It can only be the number of groups which is already stored as 3 bits above.
            break
        j += 1
        if j >= ngroups:
            raise RuntimeError('BZ_DATA_ERROR: Invalid selectorMtf value')
    bzd.selectormtf[i] = j

In [ ]:
expected = [0, 0, 1, 4, 3, 0, 0, 3, 3, 0, 0, 2, 1, 0, 3, 1, 1, 1, 2, 2, 1, 1, 2, 0, 0, 0, 1, 1, 3, 0, 3, 0, 0, 0, 0, 1, 0] + [0] * 17965

In [ ]:
self.assertListEqual(bzd.selectormtf, expected)

In [ ]:
# Undo MTF values for selectors

In [ ]:
pos = [0] * BZ_N_GROUPS

In [ ]:
pos

In [ ]:
for i in range(ngroups):
    pos[i] = i

In [ ]:
pos

In [ ]:
expected = [0, 1, 2, 3, 4, 0]

In [ ]:
self.assertListEqual(pos, expected)

In [ ]:
bzd.selector = [0] * BZ_MAX_SELECTORS

In [ ]:
for i in range(nselectors):
    v = bzd.selectormtf[i]
    tmp = pos[v]
    while v > 0:
        pos[v] = pos[v-1]
        v -= 1
    pos[0] = tmp
    bzd.selector[i] = tmp

In [ ]:
expected = [0, 0, 1, 4, 2, 2, 2, 0, 1, 1, 1, 2, 1, 1, 4, 1, 4, 1, 2, 4, 2, 4, 1, 1, 1, 1, 4, 1, 0, 0, 2, 2, 2, 2, 2, 0, 0] + [0] * 17965

In [ ]:
self.assertListEqual(bzd.selector, expected)

In [ ]:
# Coding tables

In [ ]:
bzd.len = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]

In [ ]:
for i in range(ngroups):
    curr = bzd._get_bits(5, 'BZ_X_CODING_1')
    for j in range(alphasize):
        while True:
            if curr < 1 or curr > 20:
                raise RuntimeError('BZ_DATA_ERROR: Invalid BZ_X_CODING_1')
            uc = bzd._get_bits(1, 'BZ_X_CODING_2')
            if uc == 0:
                break
            uc = bzd._get_bits(1, 'BZ_X_CODING_3')
            if uc == 0:
                curr += 1
            else:
                curr -= 1
        bzd.len[i][j] = curr

In [ ]:
expected = [[3, 3, 4, 5, 6, 5, 5, 7, 6, 5, 9, 6, 9, 7, 6, 8, 9, 9, 8, 9, 8, 8, 9, 9, 7, 8, 9, 9, 8, 7, 8, 9, 8, 9, 9, 9, 8, 9, 9, 9, 9, 8, 8, 8, 9, 9, 8, 9, 9, 9, 8, 8, 8, 9, 7, 9, 9, 9, 9, 8, 9, 8, 9, 8, 9, 8, 8, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 8, 9, 8, 9, 9, 8, 8, 8, 8, 9, 7, 8, 9, 9, 7, 8, 9, 9, 9, 8, 8, 9, 7, 8, 8, 9, 9, 8, 8, 9, 9, 8, 9, 8, 7, 9, 8, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 8, 9, 8, 9, 9, 9, 9, 9, 7, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 8, 9, 7, 9, 7, 6, 5, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 6, 6, 6, 7, 6, 7, 8, 6, 7, 7, 7, 8, 7, 8, 8, 8, 10, 8, 8, 10, 10, 9, 8, 9, 9, 9, 10, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 10, 9, 10, 9, 8, 10, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 10, 10, 9, 9, 10, 10, 10, 10, 9, 9, 9, 9, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 10, 9, 9, 9, 10, 9, 9, 10, 8, 10, 10, 10, 9, 9, 10, 10, 9, 9, 9, 9, 9, 9, 9, 10, 10, 9, 9, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 3, 3, 4, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 8, 9, 9, 9, 9, 9, 9, 9, 7, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 7, 8, 8, 7, 7, 7, 8, 7, 8, 8, 8, 8, 8, 8, 7, 7, 8, 7, 8, 7, 7, 7, 7, 8, 7, 7, 8, 7, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 8, 8, 7, 8, 8, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 8, 8, 8, 8, 7, 8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 7, 7, 7, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 8, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 8, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 4, 3, 5, 6, 5, 5, 6, 7, 6, 6, 7, 7, 7, 6, 9, 7, 9, 8, 8, 9, 9, 7, 7, 8, 8, 9, 7, 7, 9, 9, 7, 8, 8, 9, 9, 9, 9, 9, 7, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
self.assertListEqual(bzd.len, expected)

In [ ]:
# Huffman decoding tables

In [ ]:
bzd.perm = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]
bzd.base = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]
bzd.limit = [[0 for _ in range(BZ_MAX_ALPHA_SIZE)] for _ in range(BZ_N_GROUPS)]

In [ ]:
bzd.minlens = [0] * BZ_N_GROUPS

In [ ]:
for t in range(ngroups):
    minlen = 32
    maxlen = 8
    for i in range(alphasize):
        if bzd.len[t][i] > maxlen:
            maxlen = bzd.len[t][i]
        if bzd.len[t][i] < minlen:
            minlen = bzd.len[t][i]

    limit = bzd.limit[t]
    base = bzd.base[t]
    perm = bzd.perm[t]
    length = bzd.len[t]

    pp = 0
    for i in range(minlen, maxlen + 1):
        for j in range(alphasize):
            if length[j] == i:
                perm[pp] = j
                pp += 1

    for i in range(BZ_MAX_CODE_LEN):
        base[i] = 0

    for i in range(alphasize):
        base[length[i] + 1] += 1

    for i in range(BZ_MAX_CODE_LEN):
        base[i] += base[i - 1]

    for i in range(BZ_MAX_CODE_LEN):
        limit[i] = 0

    vec = 0
    for i in range(minlen, maxlen + 1):
        vec += base[i + 1] - base[i]
        limit[i] = vec - 1
        vec <<= 1

    for i in range(minlen + 1, maxlen + 1):
        base[i] = ((limit[i - 1] + 1) << 1) - base[i]

    bzd.minlens[t] = minlen

In [ ]:
expected = [[0, 1, 2, 3, 5, 6, 9, 158, 4, 8, 11, 14, 157, 7, 13, 24, 29, 54, 88, 92, 100, 112, 116, 137, 154, 156, 15, 18, 20, 21, 25, 28, 30, 32, 36, 41, 42, 43, 46, 50, 51, 52, 59, 61, 63, 65, 66, 70, 78, 80, 83, 84, 85, 86, 89, 93, 97, 98, 101, 102, 105, 106, 109, 111, 114, 125, 129, 131, 140, 148, 152, 159, 10, 12, 16, 17, 19, 22, 23, 26, 27, 31, 33, 34, 35, 37, 38, 39, 40, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 60, 62, 64, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 87, 90, 91, 94, 95, 96, 99, 103, 104, 107, 108, 110, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 130, 132, 133, 134, 135, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 153, 155, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 22, 14, 18, 20, 23, 24, 25, 27, 21, 26, 28, 29, 30, 32, 33, 37, 46, 62, 67, 77, 113, 36, 38, 39, 40, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 59, 61, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 81, 82, 87, 88, 89, 90, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 106, 107, 108, 110, 111, 117, 118, 121, 122, 123, 124, 125, 126, 127, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 31, 34, 35, 41, 54, 58, 60, 63, 79, 80, 83, 84, 85, 86, 91, 92, 101, 105, 109, 112, 114, 115, 116, 119, 120, 128, 129, 132, 156, 159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 138, 14, 26, 35, 39, 47, 68, 105, 139, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 3, 4, 5, 6, 7, 8, 9, 13, 16, 17, 18, 20, 27, 28, 30, 32, 33, 34, 35, 37, 38, 40, 42, 43, 54, 55, 56, 57, 60, 63, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 92, 96, 97, 103, 104, 105, 107, 108, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 157, 0, 1, 10, 11, 12, 14, 15, 19, 21, 22, 23, 24, 25, 26, 29, 31, 36, 39, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 58, 59, 61, 62, 64, 73, 83, 88, 89, 90, 91, 93, 94, 95, 98, 99, 100, 101, 102, 106, 109, 110, 111, 112, 113, 114, 115, 116, 133, 137, 140, 148, 156, 158, 159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 1, 3, 5, 6, 4, 7, 9, 10, 14, 8, 11, 12, 13, 16, 22, 23, 27, 28, 31, 39, 115, 18, 19, 24, 25, 32, 33, 44, 62, 95, 96, 97, 98, 99, 100, 111, 15, 17, 20, 21, 26, 29, 30, 34, 35, 36, 37, 38, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
self.assertListEqual(bzd.perm, expected)

In [ ]:
expected = [[0, 0, 0, 0, 2, 7, 22, 57, 140, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 7, 25, 69, 164, 367, 864, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 5, 14, 34, 74, 164, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 96, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 4, 11, 28, 67, 157, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
self.assertListEqual(bzd.base, expected)

In [ ]:
expected = [[0, 0, 0, 1, 4, 14, 34, 82, 211, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 5, 17, 43, 94, 202, 496, 1023, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 8, 19, 39, 89, 187, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 95, 255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 6, 16, 38, 89, 194, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
self.assertListEqual(bzd.limit, expected)

In [ ]:
expected = [3, 3, 2, 7, 2, 0]

In [ ]:
self.assertListEqual(bzd.minlens, expected)

In [ ]:
eob = bzd.ninuse + 1
nblockmax = NSIS_COMPRESS_BZIP2_LEVEL * 100_000
groupno = -1
grouppos = 0

In [ ]:
self.assertEqual(eob, 159)

In [ ]:
bzd.unzftab = [0] * 256

In [ ]:
bzd.mtfa = [0] * MTFA_SIZE
bzd.mtfbase = [0] * int(256 / MTFL_SIZE)

In [ ]:
k = MTFA_SIZE - 1

for i in reversed(range(int(256 / MTFL_SIZE))):
    for j in reversed(range(MTFL_SIZE)):
        bzd.mtfa[k] = i * MTFL_SIZE + j
        k -= 1
    bzd.mtfbase[i] = k + 1

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
self.assertListEqual(bzd.mtfa, expected)

In [ ]:
expected = [3840, 3856, 3872, 3888, 3904, 3920, 3936, 3952, 3968, 3984, 4000, 4016, 4032, 4048, 4064, 4080]

In [ ]:
self.assertListEqual(bzd.mtfbase, expected)

In [ ]:
nblock = 0

In [ ]:
gsel = 0
gminlen = 0
glimit = 0
gperm = 0
gbase = 0
zn = 0

In [ ]:
if grouppos == 0:
    groupno += 1
    if groupno >= nselectors:
        raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
    grouppos = BZ_G_SIZE
    gsel = bzd.selector[groupno]
    gminlen = bzd.minlens[gsel]
    glimit = bzd.limit[gsel]
    gperm = bzd.perm[gsel]
    gbase = bzd.base[gsel]
grouppos -= 1
zn = gminlen

In [ ]:
self.assertEqual(groupno, 0)

In [ ]:
self.assertEqual(gsel, 0)

In [ ]:
self.assertEqual(gminlen, 3)

In [ ]:
expected = [0, 0, 0, 1, 4, 14, 34, 82, 211, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(glimit, expected)

In [ ]:
expected = [0, 1, 2, 3, 5, 6, 9, 158, 4, 8, 11, 14, 157, 7, 13, 24, 29, 54, 88, 92, 100, 112, 116, 137, 154, 156, 15, 18, 20, 21, 25, 28, 30, 32, 36, 41, 42, 43, 46, 50, 51, 52, 59, 61, 63, 65, 66, 70, 78, 80, 83, 84, 85, 86, 89, 93, 97, 98, 101, 102, 105, 106, 109, 111, 114, 125, 129, 131, 140, 148, 152, 159, 10, 12, 16, 17, 19, 22, 23, 26, 27, 31, 33, 34, 35, 37, 38, 39, 40, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 60, 62, 64, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 87, 90, 91, 94, 95, 96, 99, 103, 104, 107, 108, 110, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 130, 132, 133, 134, 135, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 153, 155, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gperm, expected)

In [ ]:
expected = [0, 0, 0, 0, 2, 7, 22, 57, 140, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gbase, expected)

In [ ]:
self.assertEqual(grouppos, 49)

In [ ]:
self.assertEqual(zn, 3)

In [ ]:
zvec = bzd._get_bits(zn, 'BZ_X_MTF_1')

In [ ]:
self.assertEqual(zvec, 4)

In [ ]:
while True:
    if zn > 20:
        raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
    if zvec <= glimit[zn]:
        break
    zn += 1
    zj = bzd._get_bits(1, 'BZ_X_MTF_2')
    zvec = (zvec << 1) | zj

In [ ]:
self.assertEqual(zn, 6)

In [ ]:
self.assertEqual(zj, 0)

In [ ]:
self.assertEqual(zvec, 34)

In [ ]:
if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
    raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')

In [ ]:
nextsym = gperm[zvec - gbase[zn]]

In [ ]:
self.assertEqual(nextsym, 157)

In [ ]:
# Fourth

In [ ]:
if nblock >= nblockmax:
    raise RuntimeError('BZ_DATA_ERROR: Value nblock higher than maximum')

In [ ]:
pp = 0
nn = nextsym - 1

In [ ]:
self.assertEqual(nn, 156)

In [ ]:
# Sixth

In [ ]:
lno = int(nn / MTFL_SIZE)

In [ ]:
self.assertEqual(lno, 9)

In [ ]:
off = nn % MTFL_SIZE

In [ ]:
self.assertEqual(off, 12)

In [ ]:
pp = bzd.mtfbase[lno] + off

In [ ]:
self.assertEqual(pp, 3996)

In [ ]:
uc = bzd.mtfa[pp]

In [ ]:
self.assertEqual(uc, 156)

In [ ]:
while pp > bzd.mtfbase[lno]:
    bzd.mtfa[pp] = bzd.mtfa[pp - 1]
    pp -= 1

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
self.assertListEqual(bzd.mtfa, expected)

In [ ]:
bzd.mtfbase[lno] += 1

In [ ]:
self.assertEqual(bzd.mtfbase[lno], 3985)

In [ ]:
while lno > 0:
    bzd.mtfbase[lno] -= 1
    bzd.mtfa[bzd.mtfbase[lno]] = bzd.mtfa[bzd.mtfbase[lno - 1] + MTFL_SIZE - 1]
    lno -= 1

In [ ]:
expected = [3840, 3855, 3871, 3887, 3903, 3919, 3935, 3951, 3967, 3984, 4000, 4016, 4032, 4048, 4064, 4080]

In [ ]:
self.assertListEqual(bzd.mtfbase, expected)

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
self.assertListEqual(bzd.mtfa, expected)

In [ ]:
bzd.mtfbase[0] -= 1

In [ ]:
expected = [3839, 3855, 3871, 3887, 3903, 3919, 3935, 3951, 3967, 3984, 4000, 4016, 4032, 4048, 4064, 4080]

In [ ]:
self.assertListEqual(bzd.mtfbase, expected)

In [ ]:
bzd.mtfa[bzd.mtfbase[0]] = uc;

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 156, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
self.assertListEqual(bzd.mtfa, expected)

In [ ]:
if bzd.mtfbase[0] == 0:
    k = MTFA_SIZE - 1
    for i in reversed(range(int(256 / MTFL_SIZE))):
        for j in reversed(range(MTFL_SIZE)):
            bzd.mtfa[k] = bzd.mtfa[bzd.mtfbase[i] + j]
            k -= 1
        bzd.mtfbase[i] = k + 1

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 156, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
self.assertListEqual(bzd.mtfa, expected)

In [ ]:
expected = [3839, 3855, 3871, 3887, 3903, 3919, 3935, 3951, 3967, 3984, 4000, 4016, 4032, 4048, 4064, 4080]

In [ ]:
self.assertListEqual(bzd.mtfbase, expected)

In [ ]:
# Seventh

In [ ]:
bzd.unzftab[bzd.seqtounseq[uc]] += 1

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [ ]:
self.assertListEqual(bzd.unzftab, expected)

In [ ]:
bzd.tt = [0] * NSIS_COMPRESS_BZIP2_LEVEL * 100000

In [ ]:
bzd.tt[nblock] = bzd.seqtounseq[uc]

In [ ]:
self.assertEqual(bzd.tt[nblock], 254)

In [ ]:
nblock += 1

In [ ]:
self.assertIs(nblock, 1)

In [ ]:
if grouppos == 0:
    groupno += 1
    if groupno >= nselectors:
        raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
    grouppos = BZ_G_SIZE
    gsel = bzd.selector[groupno]
    gminlen = bzd.minlens[gsel]
    glimit = bzd.limit[gsel]
    gperm = bzd.perm[gsel]
    gbase = bzd.base[gsel]
grouppos -= 1
zn = gminlen

In [ ]:
self.assertEqual(groupno, 0)

In [ ]:
self.assertEqual(gsel, 0)

In [ ]:
self.assertEqual(gminlen, 3)

In [ ]:
expected = [0, 0, 0, 1, 4, 14, 34, 82, 211, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(glimit, expected)

In [ ]:
expected = [0, 1, 2, 3, 5, 6, 9, 158, 4, 8, 11, 14, 157, 7, 13, 24, 29, 54, 88, 92, 100, 112, 116, 137, 154, 156, 15, 18, 20, 21, 25, 28, 30, 32, 36, 41, 42, 43, 46, 50, 51, 52, 59, 61, 63, 65, 66, 70, 78, 80, 83, 84, 85, 86, 89, 93, 97, 98, 101, 102, 105, 106, 109, 111, 114, 125, 129, 131, 140, 148, 152, 159, 10, 12, 16, 17, 19, 22, 23, 26, 27, 31, 33, 34, 35, 37, 38, 39, 40, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 60, 62, 64, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 87, 90, 91, 94, 95, 96, 99, 103, 104, 107, 108, 110, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 130, 132, 133, 134, 135, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 153, 155, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gperm, expected)

In [ ]:
expected = [0, 0, 0, 0, 2, 7, 22, 57, 140, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gbase, expected)

In [ ]:
self.assertEqual(grouppos, 48)

In [ ]:
self.assertEqual(zn, 3)

In [ ]:
zvec = bzd._get_bits(zn, 'BZ_X_MTF_5')

In [ ]:
self.assertEqual(zvec, 1)

In [ ]:
while True:
    if zn > 20:
        raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
    if zvec <= glimit[zn]:
        break
    zn += 1
    zj = bzd._get_bits(1, 'BZ_X_MTF_6')
    zvec = (zvec << 1) | zj

In [ ]:
self.assertEqual(zn, 3)

In [ ]:
self.assertEqual(zj, 0)

In [ ]:
self.assertEqual(zvec, 1)

In [ ]:
if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
    raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')

In [ ]:
nextsym = gperm[zvec - gbase[zn]]

In [ ]:
self.assertEqual(nextsym, 1)

In [ ]:
# First

In [ ]:
es = -1
n = 1

In [ ]:
# Second

In [ ]:
if nextsym == BZ_RUNA:
    es += n

In [ ]:
self.assertEqual(es, -1)

In [ ]:
n = n << 1

In [ ]:
self.assertEqual(n, 2)

In [ ]:
if nextsym == BZ_RUNB:
    es += n

In [ ]:
self.assertEqual(es, 1)

In [ ]:
if grouppos == 0:
    groupno += 1
    if groupno >= nselectors:
        raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
    grouppos = BZ_G_SIZE
    gsel = bzd.selector[groupno]
    gminlen = bzd.minlens[gsel]
    glimit = bzd.limit[gsel]
    gperm = bzd.perm[gsel]
    gbase = bzd.base[gsel]
grouppos -= 1
zn = gminlen

In [ ]:
self.assertEqual(groupno, 0)

In [ ]:
self.assertEqual(gsel, 0)

In [ ]:
self.assertEqual(gminlen, 3)

In [ ]:
expected = [0, 0, 0, 1, 4, 14, 34, 82, 211, 511, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(glimit, expected)

In [ ]:
expected = [0, 1, 2, 3, 5, 6, 9, 158, 4, 8, 11, 14, 157, 7, 13, 24, 29, 54, 88, 92, 100, 112, 116, 137, 154, 156, 15, 18, 20, 21, 25, 28, 30, 32, 36, 41, 42, 43, 46, 50, 51, 52, 59, 61, 63, 65, 66, 70, 78, 80, 83, 84, 85, 86, 89, 93, 97, 98, 101, 102, 105, 106, 109, 111, 114, 125, 129, 131, 140, 148, 152, 159, 10, 12, 16, 17, 19, 22, 23, 26, 27, 31, 33, 34, 35, 37, 38, 39, 40, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 60, 62, 64, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 87, 90, 91, 94, 95, 96, 99, 103, 104, 107, 108, 110, 113, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 128, 130, 132, 133, 134, 135, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 153, 155, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gperm, expected)

In [ ]:
expected = [0, 0, 0, 0, 2, 7, 22, 57, 140, 352, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
self.assertListEqual(gbase, expected)

In [ ]:
self.assertEqual(grouppos, 47)

In [ ]:
self.assertEqual(zn, 3)

In [ ]:
zvec = bzd._get_bits(zn, 'BZ_X_MTF_3')

In [ ]:
self.assertEqual(zvec, 3)

In [ ]:
while True:
    if zn > 20:
        raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
    if zvec <= glimit[zn]:
        break
    zn += 1
    zj = bzd._get_bits(1, 'BZ_X_MTF_4')
    zvec = (zvec << 1) | zj

In [ ]:
self.assertEqual(zn, 5)

In [ ]:
self.assertEqual(zj, 0)

In [ ]:
self.assertEqual(zvec, 14)

In [ ]:
if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
    raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')

In [ ]:
nextsym = gperm[zvec - gbase[zn]]

In [ ]:
self.assertEqual(nextsym, 158)

In [ ]:
# Third

In [ ]:
es += 1

In [ ]:
self.assertEqual(es, 2)

In [ ]:
uc = bzd.seqtounseq[bzd.mtfa[bzd.mtfbase[0]]]

In [ ]:
self.assertEqual(uc, 254)

In [ ]:
bzd.unzftab[uc] += es

In [ ]:
expected = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0]

In [ ]:
self.assertListEqual(bzd.unzftab, expected)

In [ ]:
self.assertEqual(nblock, 1)

In [ ]:
print(nblock)

In [ ]:
counter = 0

In [ ]:
while es > 0:
    if nblock >= nblockmax:
        raise RuntimeError('BZ_DATA_ERROR: Value nblock too large')
    bzd.tt[nblock] = uc
    if not counter:
        self.assertIs(bzd.tt[nblock], 254)
    nblock += 1
    if not counter:
        self.assertIs(nblock, 2)
    es -= 1
    if not counter:
        self.assertEqual(es, 1)
    counter += 1

In [ ]:
expected = [254, 254, 254]

In [ ]:
actual = [z for z in bzd.tt if z]

In [ ]:
self.assertListEqual(actual, expected)

In [ ]:
# Fifth
# if nn < MTFL_SIZE:
#     pp = bzd.mtfbase[0]
#     uc = bzd.mtfa[pp + nn]
#     while nn > 0:
#         bzd.mtfa[pp + nn] = bzd.mtfa[pp + nn - 1]
#         nn -= 1
#     bzd.mtfa[pp] = uc

In [ ]:
counter = 0

In [ ]:
expected_mtfa_1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 157, 157, 156, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
expected_mtfa_2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 157, 156, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]

In [ ]:
while True:
    if nextsym == eob:
        break

    if nextsym == BZ_RUNA or nextsym == BZ_RUNB:
        es = -1
        n = 1

        while nextsym == BZ_RUNA or nextsym == BZ_RUNB:
            if nextsym == BZ_RUNA:
                es += n
            n = n << 1
            if nextsym == BZ_RUNB:
                es += n

            if grouppos == 0:
                groupno += 1
                if groupno >= nselectors:
                    raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
                grouppos = BZ_G_SIZE
                gsel = bzd.selector[groupno]
                gminlen = bzd.minlens[gsel]
                glimit = bzd.limit[gsel]
                gperm = bzd.perm[gsel]
                gbase = bzd.base[gsel]
            grouppos -= 1
            zn = gminlen

            zvec = bzd._get_bits(zn, 'BZ_X_MTF_3')

            while True:
                if zn > 20:
                    raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
                if zvec <= glimit[zn]:
                    break
                zn += 1
                zj = bzd._get_bits(1, 'BZ_X_MTF_4')
                zvec = (zvec << 1) | zj

            if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
                raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')
            nextsym = gperm[zvec - gbase[zn]]

        es += 1
        uc = bzd.seqtounseq[bzd.mtfa[bzd.mtfbase[0]]]
        bzd.unzftab[uc] += es

        while es > 0:
            if nblock >= nblockmax:
                raise RuntimeError('BZ_DATA_ERROR: Value nblock too large')
            bzd.tt[nblock] = uc
            nblock += 1
            es -= 1

    else:
        if nblock >= nblockmax:
            raise RuntimeError('BZ_DATA_ERROR: Value nblock higher than maximum')

        pp = 0
        nn = nextsym - 1

        if nn < MTFL_SIZE:
            if not counter:
                self.assertEqual(nn, 10)
            pp = bzd.mtfbase[0]
            if not counter:
                self.assertEqual(pp, 3838)
            uc = bzd.mtfa[pp + nn]
            if not counter:
                self.assertEqual(uc, 8)
            while nn > 0:
                bzd.mtfa[pp + nn] = bzd.mtfa[pp + nn - 1]
                nn -= 1
            if not counter:
                self.assertEqual(nn, 0)
                self.assertListEqual(bzd.mtfa, expected_mtfa_1)
            bzd.mtfa[pp] = uc
            if not counter:
                self.assertListEqual(bzd.mtfa, expected_mtfa_2)
            counter += 1
        else:
            lno = int(nn / MTFL_SIZE)
            off = nn % MTFL_SIZE
            pp = bzd.mtfbase[lno] + off
            uc = bzd.mtfa[pp]
            
            while pp > bzd.mtfbase[lno]:
                bzd.mtfa[pp] = bzd.mtfa[pp - 1]
                pp -= 1
            
            bzd.mtfbase[lno] += 1
            
            while lno > 0:
                bzd.mtfbase[lno] -= 1
                bzd.mtfa[bzd.mtfbase[lno]] = bzd.mtfa[bzd.mtfbase[lno - 1] + MTFL_SIZE - 1]
                lno -= 1
            
            bzd.mtfbase[0] -= 1
            
            bzd.mtfa[bzd.mtfbase[0]] = uc;
            
            if bzd.mtfbase[0] == 0:
                k = MTFA_SIZE - 1
                for i in reversed(range(int(256 / MTFL_SIZE))):
                    for j in reversed(range(MTFL_SIZE)):
                        bzd.mtfa[k] = bzd.mtfa[bzd.mtfbase[i] + j]
                        k -= 1
                    bzd.mtfbase[i] = k + 1

        bzd.unzftab[bzd.seqtounseq[uc]] += 1
        bzd.tt[nblock] = bzd.seqtounseq[uc]
        nblock += 1

        if grouppos == 0:
            groupno += 1
            if groupno >= nselectors:
                raise RuntimeError('BZ_DATA_ERROR: Number of groups too large')
            grouppos = BZ_G_SIZE
            gsel = bzd.selector[groupno]
            gminlen = bzd.minlens[gsel]
            glimit = bzd.limit[gsel]
            gperm = bzd.perm[gsel]
            gbase = bzd.base[gsel]
        grouppos -= 1
        zn = gminlen

        zvec = bzd._get_bits(zn, 'BZ_X_MTF_5')

        while True:
            if zn > 20:
                raise RuntimeError('BZ_DATA_ERROR: Value zv too large')
            if zvec <= glimit[zn]:
                break
            zn += 1
            zj = bzd._get_bits(1, 'BZ_X_MTF_6')
            zvec = (zvec << 1) | zj

        if zvec - gbase[zn] < 0 or zvec - gbase[zn] >= BZ_MAX_ALPHA_SIZE:
            raise RuntimeError('BZ_DATA_ERROR: Value zvec outside of acceptable range')
        
        nextsym = gperm[zvec - gbase[zn]]

In [ ]:
tt = [z for z in bzd.tt if z]

In [ ]:
self.assertEqual(len(tt), 1735)

In [ ]:
expected = [254, 254, 254, 255, 255, 255, 255, 8, 11, 2, 1, 96, 17, 4, 255, 1, 1, 2, 251, 1, 1, 255, 1, 1, 23, 39, 251, 251, 251, 251, 251, 251, 251, 138, 79, 188, 214, 1, 1, 104, 48, 103, 51, 51, 51, 51, 51, 101, 115, 115, 3, 106, 116, 101, 114, 242, 242, 4, 4, 105, 3, 77, 66, 75, 103, 100, 58, 107, 1, 128, 4, 121, 84, 101, 1, 1, 4, 101, 100, 46, 108, 46, 58, 4, 100, 114, 114, 32, 109, 115, 108, 62, 114, 115, 32, 32, 46, 32, 32, 116, 1, 32, 32, 71, 63, 32, 74, 71, 101, 34, 49, 101, 11, 58, 32, 32, 58, 46, 114, 103, 32, 101, 112, 32, 116, 116, 82, 21, 253, 11, 74, 10, 20, 20, 20, 119, 176, 119, 10, 8, 198, 6, 119, 250, 233, 244, 119, 62, 101, 107, 113, 74, 74, 1, 3, 5, 2, 22, 2, 52, 32, 1, 144, 144, 144, 10, 3, 3, 48, 2, 4, 5, 4, 2, 5, 5, 1, 128, 2, 5, 1, 5, 128, 128, 1, 2, 2, 2, 5, 2, 1, 3, 2, 2, 2, 2, 2, 1, 3, 1, 4, 4, 4, 1, 3, 1, 4, 3, 4, 2, 3, 2, 3, 255, 1, 2, 32, 32, 10, 92, 34, 32, 32, 255, 13, 13, 13, 13, 13, 13, 10, 32, 10, 114, 44, 2, 255, 58, 58, 58, 58, 58, 58, 58, 58, 101, 58, 58, 58, 108, 108, 46, 46, 58, 60, 119, 116, 107, 107, 111, 58, 33, 121, 109, 110, 110, 110, 116, 107, 58, 58, 58, 107, 110, 108, 83, 115, 46, 110, 121, 101, 108, 116, 101, 44, 116, 101, 101, 108, 111, 114, 103, 114, 97, 117, 58, 111, 103, 115, 101, 101, 116, 114, 103, 116, 101, 101, 101, 108, 108, 111, 111, 111, 111, 111, 102, 111, 101, 101, 100, 101, 101, 114, 101, 44, 110, 101, 100, 44, 111, 111, 111, 107, 107, 116, 110, 112, 116, 116, 116, 112, 116, 103, 114, 116, 116, 115, 104, 101, 108, 116, 121, 111, 101, 100, 100, 109, 117, 116, 112, 117, 116, 114, 114, 115, 115, 115, 97, 101, 2, 32, 32, 66, 110, 110, 110, 114, 110, 114, 101, 110, 101, 46, 114, 108, 114, 46, 101, 51, 108, 49, 50, 108, 51, 108, 116, 49, 46, 118, 110, 108, 108, 108, 100, 100, 101, 114, 101, 116, 103, 116, 114, 114, 101, 103, 116, 67, 32, 114, 32, 38, 32, 10, 32, 38, 92, 32, 32, 92, 32, 115, 115, 84, 32, 32, 110, 109, 32, 32, 32, 83, 10, 38, 32, 32, 32, 32, 32, 32, 32, 92, 111, 121, 38, 32, 32, 92, 111, 10, 73, 78, 69, 32, 108, 32, 83, 92, 92, 92, 32, 32, 32, 92, 116, 92, 115, 101, 58, 108, 101, 110, 116, 32, 116, 108, 112, 112, 66, 114, 114, 32, 118, 116, 116, 103, 112, 110, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 114, 114, 78, 67, 67, 32, 32, 32, 119, 119, 111, 119, 119, 116, 100, 101, 101, 110, 108, 108, 108, 108, 32, 108, 97, 112, 101, 65, 97, 97, 110, 110, 97, 105, 105, 105, 105, 101, 101, 32, 32, 32, 32, 32, 101, 32, 32, 105, 101, 101, 101, 101, 101, 97, 97, 101, 101, 114, 101, 110, 110, 101, 101, 101, 32, 32, 108, 108, 108, 108, 108, 108, 108, 108, 38, 32, 111, 32, 110, 115, 120, 120, 120, 109, 120, 120, 108, 115, 99, 104, 104, 116, 104, 118, 104, 108, 104, 104, 112, 116, 104, 99, 114, 104, 120, 117, 108, 115, 116, 108, 108, 114, 114, 114, 114, 114, 104, 104, 100, 108, 108, 108, 108, 108, 116, 116, 116, 116, 114, 112, 99, 115, 83, 83, 115, 83, 68, 68, 116, 82, 112, 114, 114, 110, 110, 110, 110, 82, 114, 100, 100, 104, 100, 100, 100, 100, 108, 100, 100, 108, 102, 32, 86, 108, 108, 108, 108, 114, 84, 84, 84, 84, 68, 100, 68, 108, 108, 108, 108, 82, 83, 83, 46, 46, 46, 46, 46, 46, 78, 78, 111, 102, 105, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 111, 111, 111, 111, 105, 110, 110, 110, 110, 110, 110, 110, 110, 97, 73, 111, 111, 115, 116, 116, 116, 116, 116, 116, 116, 116, 99, 67, 116, 116, 83, 110, 108, 108, 108, 108, 77, 100, 100, 97, 102, 102, 102, 102, 70, 70, 70, 70, 119, 97, 97, 32, 32, 97, 32, 116, 87, 114, 108, 115, 110, 119, 116, 116, 116, 46, 119, 110, 32, 32, 32, 32, 32, 110, 110, 110, 32, 32, 32, 110, 110, 32, 32, 110, 32, 116, 116, 116, 116, 116, 116, 115, 116, 116, 107, 108, 107, 97, 68, 68, 117, 104, 119, 68, 110, 114, 114, 114, 99, 99, 99, 99, 99, 99, 99, 115, 83, 101, 108, 108, 108, 108, 108, 97, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 105, 108, 108, 108, 108, 112, 111, 111, 111, 111, 111, 111, 111, 111, 105, 105, 105, 105, 98, 101, 101, 101, 101, 101, 108, 108, 105, 105, 105, 105, 101, 101, 112, 112, 67, 99, 67, 67, 108, 67, 97, 97, 97, 97, 97, 97, 105, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 111, 117, 67, 108, 105, 105, 108, 111, 97, 101, 97, 97, 111, 111, 109, 109, 101, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 105, 111, 105, 111, 97, 111, 105, 111, 105, 111, 111, 111, 105, 97, 117, 97, 97, 105, 111, 111, 111, 111, 105, 105, 105, 105, 105, 105, 105, 105, 105, 101, 105, 117, 117, 117, 85, 85, 117, 105, 103, 97, 111, 73, 105, 105, 73, 105, 105, 73, 105, 105, 105, 105, 105, 105, 105, 105, 105, 73, 73, 105, 105, 105, 105, 73, 101, 97, 97, 101, 111, 101, 101, 101, 101, 105, 84, 116, 116, 116, 116, 84, 116, 116, 116, 116, 116, 116, 116, 78, 78, 98, 77, 32, 115, 115, 83, 83, 114, 114, 70, 70, 102, 102, 102, 102, 102, 102, 102, 116, 67, 67, 67, 67, 99, 99, 99, 99, 99, 109, 105, 105, 105, 32, 100, 105, 105, 109, 105, 105, 112, 112, 112, 112, 105, 99, 98, 116, 32, 32, 67, 32, 114, 114, 114, 114, 102, 110, 67, 98, 108, 114, 111, 110, 121, 121, 121, 109, 104, 108, 100, 114, 114, 117, 111, 105, 117, 83, 83, 101, 105, 121, 112, 111, 115, 109, 109, 109, 109, 109, 109, 105, 109, 117, 111, 79, 116, 111, 101, 105, 101, 105, 105, 101, 111, 111, 111, 101, 111, 101, 111, 111, 101, 79, 101, 101, 101, 101, 101, 101, 116, 116, 103, 103, 97, 111, 97, 97, 67, 99, 32, 105, 101, 114, 32, 112, 116, 119, 119, 119, 80, 80, 114, 114, 114, 114, 99, 66, 38, 117, 69, 69, 69, 101, 111, 101, 97, 111, 114, 116, 116, 101, 101, 110, 108, 108, 105, 116, 116, 116, 116, 101, 101, 119, 111, 119, 119, 32, 32, 105, 115, 114, 32, 108, 111, 105, 101, 101, 101, 101, 101, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 110, 32, 121, 101, 117, 32, 115, 115, 115, 115, 120, 102, 99, 99, 110, 117, 99, 114, 99, 99, 120, 110, 110, 114, 39, 39, 105, 111, 99, 99, 110, 102, 97, 101, 101, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 101, 101, 105, 101, 101, 112, 97, 115, 32, 32, 32, 32, 32, 32, 32, 32, 111, 32, 115, 105, 97, 105, 110, 97, 97, 97, 97, 97, 112, 112, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 115, 115, 117, 120, 120, 83, 32, 101, 110, 110, 110, 110, 101, 101, 102, 102, 32, 111, 111, 111, 110, 113, 78, 32, 92, 92, 92, 32, 116, 116, 114, 67, 67, 112, 79, 32, 97, 111, 111, 32, 32, 116, 116, 32, 111, 92, 32, 32, 32, 32, 111, 111, 111, 101, 101, 101, 101, 101, 101, 101, 101, 69, 69, 102, 112, 114, 114, 97, 32, 32, 32, 116, 83, 3, 2, 2, 1, 2, 3, 3, 3, 3, 3, 3, 3, 1, 1, 158, 14, 251, 194, 108, 6, 44, 34, 64, 73, 91, 122, 144, 177, 14, 241, 219, 219, 219, 69, 43, 44, 251, 27, 59, 95, 97, 100, 103, 106, 237, 244, 3, 16, 246, 18, 116, 144, 200, 230, 238, 9, 235, 14, 36, 160, 179, 190, 226, 255, 20, 33, 47, 57, 69, 76, 8, 255, 8, 255, 255, 255, 255, 3, 8, 68, 120, 7, 128, 8, 255, 255, 255, 46, 40, 16, 6, 196, 196, 196, 172, 172, 172, 129, 128, 128, 128, 129, 4, 36, 255, 2, 109, 15, 255, 145, 6, 255, 16, 4, 25, 128, 128, 32, 25, 128, 128, 128, 36, 2, 5, 255, 5, 5, 128, 128, 128, 128, 22, 158, 158, 149, 128, 130, 187, 187, 149, 149, 149, 149, 2, 31, 31, 31, 177, 177, 177, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 124, 29, 29, 29, 1, 1, 1, 10, 255, 255, 255, 255, 255, 255, 1, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 217, 216, 213, 197, 199, 217, 255, 255, 255, 255, 255, 255, 255, 255, 219, 219, 204, 255, 219, 218, 212, 211, 211, 200, 203, 211, 205, 206, 207, 217, 197, 210, 198, 201, 208, 214, 215, 217, 209, 202, 209, 4, 30, 241]

In [ ]:
self.assertListEqual(tt, expected)

In [ ]:
if bzd.origptr < 0:
    raise RuntimeError('BZ_DATA_ERROR: Orig ptr is negative')

In [ ]:
if bzd.origptr > 10 + NSIS_COMPRESS_BZIP2_LEVEL * 100000:
    raise RuntimeError('BZ_DATA_ERROR: Orig ptr is too large')

In [ ]:
cftab = [0] * 257

In [ ]:
for i in range(1, 257):
    cftab[i] = bzd.unzftab[i - 1] + cftab[i - 1]

In [ ]:
expected = [0, 1857, 1888, 1915, 1936, 1952, 1962, 1966, 1967, 1973, 1974, 1984, 1987, 1987, 1993, 1996, 1997, 2000, 2001, 2002, 2002, 2006, 2007, 2009, 2010, 2010, 2012, 2012, 2013, 2013, 2016, 2017, 2020, 2158, 2160, 2163, 2163, 2166, 2166, 2172, 2175, 2176, 2176, 2176, 2177, 2183, 2183, 2201, 2202, 2204, 2207, 2208, 2215, 2216, 2216, 2216, 2216, 2216, 2217, 2239, 2240, 2241, 2241, 2243, 2244, 2245, 2246, 2250, 2268, 2276, 2284, 2290, 2292, 2292, 2301, 2305, 2306, 2307, 2310, 2317, 2321, 2323, 2323, 2327, 2343, 2351, 2353, 2354, 2355, 2355, 2355, 2355, 2356, 2370, 2370, 2370, 2371, 2372, 2435, 2439, 2466, 2491, 2603, 2623, 2637, 2652, 2729, 2731, 2741, 2832, 2850, 2930, 3017, 3042, 3044, 3106, 3164, 3276, 3294, 3297, 3316, 3327, 3337, 3338, 3338, 3339, 3339, 3339, 3339, 3357, 3359, 3360, 3360, 3360, 3360, 3360, 3360, 3360, 3360, 3361, 3361, 3361, 3361, 3361, 3361, 3366, 3367, 3367, 3367, 3367, 3372, 3372, 3372, 3372, 3372, 3372, 3372, 3372, 3372, 3375, 3375, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3376, 3379, 3379, 3379, 3379, 3380, 3384, 3384, 3385, 3385, 3385, 3385, 3385, 3385, 3385, 3385, 3387, 3388, 3388, 3389, 3389, 3389, 3389, 3390, 3390, 3393, 3395, 3397, 3398, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3410, 3411, 3414, 3415, 3416, 3418, 3419, 3420, 3424, 3425, 3431, 3431, 3431, 3431, 3431, 3431, 3431, 3432, 3432, 3432, 3432, 3433, 3433, 3433, 3434, 3434, 3435, 3435, 3436, 3437, 3437, 3437, 3439, 3441, 3441, 3443, 3443, 3444, 3444, 3444, 3444, 3445, 3455, 3455, 3456, 3459, 3592]

In [ ]:
self.assertListEqual(cftab, expected)

In [ ]:
for i in range(nblock):
    uc = bzd.tt[i] & 0xff
    bzd.tt[cftab[uc]] |= (i << 8)
    cftab[uc] += 1

In [ ]:
self.assertEqual(nblock, 3592)

In [ ]:
tt = [z for z in bzd.tt if z]

In [ ]:
self.assertEqual(len(tt), 3591)

In [ ]:
expected = [6910, 7166, 7422, 7679, 7935, 8191, 8447, 12296, 14859, 15106, 15361, 16992, 17169, 18436, 18943, 18945, 19201, 19458, 19963, 20225, 20481, 20991, 21249, 21505, 21783, 39719, 41216, 41472, 41728, 43008, 43520, 43776, 44544, 46331, 46587, 46843, 47099, 47355, 47611, 47867, 48010, 50767, 51132, 51414, 51713, 51969, 52584, 52784, 52992, 53351, 53555, 53811, 54067, 54323, 54579, 55141, 55411, 55923, 56064, 56320, 56576, 56835, 57194, 57460, 57701, 57970, 58112, 58368, 58866, 59122, 59396, 59652, 59904, 60160, 61184, 61440, 61696, 61952, 62313, 62464, 63488, 64000, 64515, 65024, 65280, 65536, 65869, 66114, 66379, 66919, 67428, 67642, 67947, 68097, 68736, 68868, 69497, 69716, 69989, 70145, 70401, 70660, 71013, 71524, 71726, 72044, 72238, 72506, 72708, 73316, 73586, 73842, 74016, 74349, 74611, 74860, 75070, 75378, 75635, 75808, 76064, 76334, 76576, 76832, 77428, 77569, 77856, 78112, 78407, 78655, 78880, 79690, 80455, 80741, 80930, 81201, 81509, 81675, 81978, 82208, 82464, 82746, 82990, 83570, 83815, 84000, 84325, 84848, 85024, 85364, 85620, 86098, 87317, 88317, 88587, 88832, 89162, 90634, 90900, 91156, 91412, 92672, 92928, 93184, 93559, 94128, 94839, 94986, 95488, 95752, 96256, 96512, 96966, 97030, 97280, 98167, 99322, 99561, 99828, 99959, 100864, 101120, 101376, 101632, 101888, 102144, 102400, 102656, 103486, 103781, 104043, 104305, 104522, 104778, 104961, 105219, 105477, 105730, 105984, 106240, 106496, 106774, 107008, 107264, 107522, 107776, 108032, 108800, 109056, 109312, 109568, 109824, 110080, 110336, 110644, 110848, 111104, 111392, 111616, 111872, 112128, 112384, 112640, 112896, 113408, 113664, 113920, 115200, 115456, 115712, 115968, 116225, 116480, 116736, 116992, 117248, 117904, 118160, 118416, 118528, 118784, 119040, 119296, 119552, 121344, 121610, 123139, 123395, 123648, 123952, 124416, 124930, 125184, 125444, 125696, 125952, 126208, 134144, 135680, 135936, 160773, 161280, 161540, 162304, 162560, 162816, 164096, 164354, 165632, 165888, 170501, 171008, 171264, 171776, 172032, 172288, 172544, 173056, 173317, 177664, 177920, 178176, 178432, 179200, 179712, 180225, 180736, 180992, 181760, 182016, 182272, 182528, 183040, 183296, 184320, 184832, 185088, 185600, 186112, 186368, 186624, 187008, 187904, 188160, 188672, 188928, 189184, 189440, 189696, 189954, 190213, 194304, 194561, 194821, 195072, 196352, 197376, 197632, 197888, 198144, 198656, 198912, 199424, 200960, 201216, 201600, 201984, 202240, 202496, 202752, 203136, 203264, 203520, 204544, 206336, 207361, 209408, 225538, 239874, 276738, 276997, 300544, 300802, 326401, 331008, 439811, 451840, 454656, 454912, 455170, 455426, 455682, 455938, 456194, 457472, 457728, 457984, 458240, 458497, 458755, 460545, 461316, 461568, 461824, 462080, 462336, 462596, 463360, 463620, 463873, 464128, 464384, 464643, 464896, 465152, 465409, 465664, 465920, 466176, 466432, 466688, 466948, 467203, 467456, 467716, 467970, 468227, 468480, 468736, 468992, 469248, 469506, 469763, 470271, 470272, 470528, 470784, 471040, 471296, 471552, 471808, 472064, 472321, 472578, 472832, 473088, 473344, 473600, 473856, 474112, 474368, 474624, 474880, 475136, 476416, 477184, 477440, 477696, 477952, 478208, 478464, 478720, 481536, 481824, 482080, 483072, 483840, 486144, 486400, 486656, 486912, 487168, 488192, 488448, 488704, 488960, 489216, 489472, 489728, 489984, 490496, 490752, 493066, 493312, 493568, 493824, 494172, 494370, 494624, 494880, 495104, 495360, 495872, 496128, 496384, 496640, 496896, 498944, 499456, 499967, 501504, 501760, 502016, 502528, 502784, 503040, 503296, 505088, 506125, 506381, 506637, 506893, 507149, 507405, 507648, 508416, 508682, 508960, 509450, 509810, 509996, 510208, 510720, 510976, 511232, 511490, 511744, 512255, 512256, 512512, 513280, 514048, 514560, 516096, 517178, 517434, 517690, 517946, 518202, 518458, 518714, 518970, 519269, 519482, 519738, 519994, 520556, 520812, 521006, 521262, 521530, 521788, 522103, 522356, 522603, 522859, 523119, 523322, 523553, 523897, 524141, 524398, 524654, 524910, 525056, 525428, 525675, 525882, 526138, 526394, 526592, 526848, 527211, 527470, 527980, 528211, 528499, 528686, 529006, 529273, 529509, 529772, 530036, 530277, 530476, 530804, 531045, 531301, 531564, 531823, 532082, 532327, 532594, 532833, 533109, 533306, 533615, 533863, 534131, 534373, 534629, 534900, 535154, 535399, 535668, 535909, 536165, 536421, 536684, 537452, 537711, 537967, 538223, 538479, 538735, 538982, 539247, 539493, 539749, 540004, 540261, 540517, 540786, 541029, 541228, 541550, 541797, 542052, 542252, 542575, 542831, 543087, 543339, 543595, 543860, 544110, 544368, 544628, 544884, 545140, 545392, 545652, 545895, 546162, 546420, 546676, 546931, 547176, 547429, 547692, 547956, 548217, 548463, 548709, 548964, 549220, 549485, 549749, 550004, 550256, 550517, 550772, 551026, 551282, 551539, 551795, 552051, 552545, 552704, 553061, 553216, 553728, 553986, 554528, 554752, 555008, 555264, 555552, 555842, 556398, 556654, 556800, 557056, 557422, 557682, 557934, 558194, 558336, 558592, 558949, 559214, 559461, 559662, 559986, 560236, 560498, 560686, 560997, 561203, 561516, 561713, 561970, 562284, 562483, 562796, 563060, 563456, 563761, 564224, 564480, 564782, 564992, 565760, 566016, 566272, 566646, 566784, 567296, 567662, 567916, 568172, 568428, 568676, 568932, 569189, 569458, 569701, 569972, 570215, 570484, 570738, 570994, 571237, 571495, 571648, 571904, 572160, 572416, 572788, 572995, 573440, 573728, 573952, 574322, 574464, 574752, 574976, 575232, 575526, 575776, 576000, 576256, 576512, 576768, 577034, 577280, 577536, 577824, 578086, 578396, 578560, 578848, 579072, 579328, 579616, 579840, 580188, 580352, 580608, 580864, 581120, 581408, 581747, 582003, 582144, 582400, 582740, 582912, 583168, 583424, 583680, 583936, 584192, 584448, 584736, 584992, 585326, 585581, 585760, 586016, 586272, 586835, 587018, 587302, 587552, 587808, 588064, 588320, 588576, 588800, 589056, 589312, 589824, 590112, 590624, 590940, 591215, 591360, 591737, 591910, 592160, 592384, 592640, 592896, 593152, 593440, 593664, 593920, 594268, 594944, 595311, 595466, 595785, 596046, 596293, 596480, 596736, 596992, 597280, 597504, 597760, 598016, 598272, 598528, 598784, 599040, 599404, 599584, 599891, 600064, 600412, 600668, 600924, 601120, 601376, 601632, 601856, 602204, 602484, 602716, 602880, 603763, 604005, 604218, 604780, 605029, 605294, 605556, 606976, 607264, 607604, 607852, 608112, 608368, 608578, 608882, 609138, 609312, 609654, 609908, 610164, 610407, 610672, 610926, 611188, 611444, 611700, 611956, 612212, 612468, 612724, 612980, 613236, 613492, 613748, 614004, 614260, 614516, 614772, 615028, 615284, 615540, 615796, 616052, 616308, 616564, 616820, 617074, 617330, 617550, 617795, 618051, 618272, 618528, 618784, 619127, 619383, 619631, 619895, 620151, 620404, 620644, 620901, 621157, 621422, 621676, 621932, 622188, 622444, 622624, 622956, 623104, 623457, 623728, 623973, 624193, 624481, 624737, 625006, 625262, 625505, 625769, 626025, 626281, 626537, 626789, 627045, 627232, 627488, 627744, 628000, 628256, 628581, 628768, 629024, 629353, 629605, 629861, 630117, 630373, 630629, 630881, 631137, 631397, 631653, 631922, 632165, 632430, 632686, 632933, 633189, 633445, 633632, 633888, 634220, 634476, 634732, 634988, 635244, 635500, 635756, 636012, 636198, 636448, 636783, 636960, 637294, 637440, 637811, 638072, 638328, 638584, 638829, 639096, 639352, 639852, 640115, 640355, 640616, 640872, 641140, 641384, 641654, 641896, 642156, 642408, 642664, 642928, 643188, 643432, 643683, 643954, 644200, 644472, 644725, 644972, 645235, 645492, 645740, 645996, 646258, 646514, 646770, 647026, 647282, 647528, 647784, 648036, 648300, 648556, 648812, 649068, 649324, 649588, 649844, 650100, 650356, 650610, 650864, 651107, 651379, 651603, 651859, 652147, 652371, 652612, 652868, 653172, 653394, 653680, 653938, 654194, 654446, 654702, 654958, 655214, 655442, 655730, 655972, 656228, 656488, 656740, 656996, 657252, 657508, 657772, 658020, 658276, 658540, 658790, 658976, 659286, 659564, 659820, 660076, 660332, 660594, 660820, 661076, 661332, 661588, 661828, 662116, 662340, 662636, 662892, 663148, 663404, 663634, 663891, 664147, 664366, 664622, 664878, 665134, 665390, 665646, 665934, 666190, 666479, 666726, 666985, 667168, 667424, 667680, 667936, 668192, 668448, 668704, 668960, 669216, 669472, 669728, 669984, 670319, 670575, 670831, 671087, 671337, 671854, 672110, 672366, 672622, 672878, 673134, 673390, 673646, 673889, 674121, 674415, 674671, 674816, 675072, 675443, 675700, 675956, 676212, 676468, 676724, 676980, 677236, 677492, 677731, 677955, 678260, 678516, 678739, 679022, 679276, 679532, 679788, 680044, 680269, 680548, 680804, 681057, 681318, 681574, 681830, 682086, 682310, 682566, 682822, 683078, 683383, 683617, 683873, 684064, 684320, 684641, 684832, 685172, 685399, 685682, 685932, 686195, 686446, 686711, 686964, 687220, 687476, 687662, 687991, 688238, 688416, 688672, 688928, 689184, 689440, 689774, 690030, 690286, 690464, 690720, 690976, 691310, 691566, 691744, 692000, 692334, 692512, 692852, 693108, 693364, 693620, 693876, 694132, 694387, 694644, 694900, 695147, 695404, 695659, 695905, 696132, 696388, 696693, 696936, 697207, 697412, 697710, 697970, 698226, 698482, 698624, 698880, 699235, 699747, 700003, 700259, 700515, 700771, 701027, 701299, 701523, 701797, 702060, 702316, 702572, 702828, 703084, 703329, 703596, 703852, 704108, 704364, 704620, 704876, 705132, 705388, 705644, 705900, 706156, 706412, 706665, 706924, 707180, 707436, 707692, 707952, 708207, 708463, 708719, 708975, 709231, 709487, 709743, 709999, 710249, 710505, 710761, 711017, 711266, 711525, 711781, 712037, 712293, 712549, 712812, 713068, 713321, 713577, 713833, 714089, 714341, 714597, 714864, 715120, 715331, 715619, 715843, 716099, 716396, 716611, 716897, 717153, 717409, 717665, 717921, 718177, 718441, 718689, 718945, 719201, 719457, 719713, 719969, 720225, 720481, 720737, 720993, 721249, 721505, 721761, 722017, 722273, 722529, 722785, 723055, 723317, 723523, 723820, 724073, 724329, 724588, 724736, 725103, 725345, 725605, 725857, 726113, 726383, 726639, 726893, 727149, 727397, 727663, 727919, 728175, 728431, 728687, 728943, 729199, 729344, 729711, 729967, 730223, 730479, 730729, 730991, 731241, 731503, 731745, 732015, 732265, 732527, 732777, 733039, 733295, 733551, 733801, 734049, 734325, 734561, 734817, 735081, 735343, 735599, 735855, 736111, 736361, 736617, 736873, 737129, 737385, 737641, 737897, 738153, 738409, 738661, 738921, 739189, 739445, 739701, 739925, 740181, 740469, 740713, 740967, 741217, 741487, 741705, 741993, 742249, 742473, 742761, 743017, 743241, 743529, 743785, 744041, 744297, 744553, 744809, 745065, 745321, 745577, 745801, 746057, 746345, 746601, 746857, 747113, 747337, 747621, 747873, 748129, 748389, 748655, 748901, 749157, 749413, 749669, 749929, 750164, 750452, 750708, 750964, 751220, 751444, 751732, 751988, 752244, 752500, 752756, 753012, 753268, 753486, 753742, 754018, 754253, 754464, 754803, 755059, 755283, 755539, 755826, 756082, 756294, 756550, 756838, 757094, 757350, 757606, 757862, 758118, 758374, 758644, 758851, 759107, 759363, 759619, 759907, 760163, 760419, 760675, 760931, 761197, 761449, 761705, 761961, 762144, 762468, 762729, 762985, 763245, 763497, 763753, 764016, 764272, 764528, 764784, 765033, 765283, 765538, 765812, 765984, 766240, 766531, 766752, 767090, 767346, 767602, 767858, 768102, 768366, 768579, 768866, 769132, 769394, 769647, 769902, 770169, 770425, 770681, 770925, 771176, 771436, 771684, 771954, 772210, 772469, 772719, 772969, 773237, 773459, 773715, 773989, 774249, 774521, 774768, 775023, 775283, 775533, 775789, 776045, 776301, 776557, 776813, 777065, 777325, 777589, 777839, 778063, 778356, 778607, 779109, 779369, 779621, 779881, 780137, 780389, 780655, 780911, 781167, 781413, 781679, 781925, 782191, 782447, 782693, 782927, 783205, 783461, 783717, 783973, 784229, 784485, 784756, 785012, 785255, 785511, 785761, 786031, 786273, 786529, 786755, 787043, 787232, 787561, 787813, 788082, 788256, 788592, 788852, 789111, 789367, 789623, 789840, 790096, 790386, 790642, 790898, 791154, 791395, 791618, 791846, 792181, 792389, 792645, 792901, 793189, 793455, 793701, 793953, 794223, 794482, 794740, 794996, 795237, 795493, 795758, 796012, 796268, 796521, 796788, 797044, 797300, 797556, 797797, 798053, 798327, 798575, 798839, 799095, 799264, 799520, 799849, 800115, 800370, 800544, 800876, 801135, 801385, 801637, 801893, 802149, 802405, 802661, 802926, 803182, 803438, 803694, 803950, 804206, 804462, 804718, 804974, 805230, 805486, 805742, 805998, 806254, 806510, 806766, 807022, 807278, 807534, 807790, 808046, 808302, 808558, 808736, 809081, 809317, 809589, 809760, 810099, 810355, 810611, 810867, 811128, 811366, 811619, 811875, 812142, 812405, 812643, 812914, 813155, 813411, 813688, 813934, 814190, 814450, 814631, 814887, 815209, 815471, 815715, 815971, 816238, 816486, 816737, 816997, 817253, 817523, 817779, 818035, 818291, 818547, 818803, 819059, 819315, 819571, 819827, 820083, 820339, 820595, 820851, 821107, 821363, 821619, 821875, 822131, 822387, 822643, 822899, 823155, 823411, 823653, 823909, 824169, 824421, 824677, 824944, 825185, 825459, 825632, 825888, 826144, 826400, 826656, 826912, 827168, 827424, 827759, 827936, 828275, 828521, 828769, 829033, 829294, 829537, 829793, 830049, 830305, 830561, 830832, 831088, 831264, 831520, 831776, 832032, 832288, 832544, 832800, 833056, 833312, 833568, 833824, 834163, 834419, 834677, 834936, 835192, 835411, 835616, 835941, 836206, 836462, 836718, 836974, 837221, 837477, 837734, 837990, 838176, 838400, 838767, 839023, 839279, 839534, 839793, 840014, 840224, 840540, 840796, 841052, 841248, 841588, 841844, 842098, 842307, 842563, 842864, 843087, 843296, 843617, 843887, 845167, 845344, 845600, 845940, 846196, 846368, 846703, 846940, 847136, 847392, 847648, 847904, 848239, 848495, 848751, 848997, 849253, 849509, 849765, 850021, 850277, 850533, 850789, 851013, 851269, 851456, 851814, 852080, 852338, 852594, 852833, 853024, 853280, 853536, 853876, 854099, 854528, 854784, 856832, 858112, 858368, 858624, 858880, 859139, 859394, 859650, 859905, 860160, 860416, 860672, 860928, 861184, 861440, 861698, 861955, 862211, 862467, 862723, 862979, 863232, 863491, 863747, 864000, 864256, 864512, 864768, 865024, 866304, 866561, 866817, 867072, 867328, 867584, 869120, 869376, 869888, 870144, 870400, 872192, 872960, 873984, 874496, 875264, 875776, 878336, 878592, 878848, 879104, 879360, 879616, 880128, 880384, 880640, 880896, 881152, 881408, 881664, 881920, 882176, 882432, 882688, 882944, 883200, 883456, 883712, 883968, 884224, 884480, 884736, 884992, 885248, 917504, 917760, 918272, 918528, 918784, 919040, 2718, 3854, 4347, 5058, 5120, 5740, 5894, 11264, 11520, 23808, 25344, 25600, 32000, 49664, 59180, 72994, 79936, 85833, 87899, 91770, 92304, 94641, 96014, 103153, 457216, 462848, 463104, 877787, 878043, 878299, 887040, 2373, 4395, 50432, 52268, 64507, 68379, 79163, 86367, 86625, 86884, 87655, 89344, 89600, 89856, 90112, 90368, 98666, 100333, 103412, 124160, 161792, 456704, 456960, 459008, 563200, 606720, 865280, 15619, 20992, 49920, 62992, 63478, 88338, 92020, 95376, 97992, 99046, 100590, 456448, 459264, 459520, 459776, 460032, 460288, 460800, 461056, 490240, 505344, 3337, 17920, 18176, 24320, 25856, 27648, 65003, 67086, 92452, 93856, 94387, 97726, 98530, 553472, 590591, 917248, 50176, 66580, 69153, 71215, 79417, 80197, 87116, 639488, 699392, 778752, 44296, 476928, 515072, 573184, 508160, 2047, 43272, 502527, 504063, 505855, 510719, 495616, 40195, 42760, 62788, 113152, 121856, 122368, 182784, 192512, 199936, 879872, 2168, 35079, 39424, 119808, 120064, 120448, 120576, 120832, 121088, 475648, 481032, 497408, 565248, 491008, 514816, 589568, 3072, 491520, 40448, 40959, 41215, 499968, 39167, 51502, 854272, 6184, 594432, 604432, 484614, 876996, 877252, 877508, 918016, 865708, 865964, 866220, 28672, 30464, 30720, 31232, 31488, 32256, 32512, 33280, 35584, 35840, 37120, 37888, 55681, 108288, 108544, 114688, 114944, 122112, 162048, 163072, 179456, 180480, 181504, 183552, 184576, 185344, 187136, 190464, 190720, 191488, 191744, 192000, 193024, 193280, 193536, 193792, 194048, 195328, 195584, 197120, 198400, 201856, 204032, 205568, 205824, 206080, 209664, 211712, 220672, 220928, 221184, 225024, 229632, 229888, 230144, 230400, 230656, 231168, 231424, 236032, 236288, 238848, 239360, 261120, 269312, 269568, 269824, 270080, 270336, 270592, 270848, 271104, 271360, 271616, 271872, 272128, 285952, 286336, 286848, 290304, 290560, 290816, 291072, 291328, 292352, 292608, 292864, 293632, 293888, 294400, 356096, 363776, 367616, 367872, 368384, 388608, 389632, 400640, 400896, 401920, 410112, 411136, 427008, 427264, 427520, 427776, 428032, 428288, 428544, 428800, 429312, 432640, 432896, 433152, 433408, 433664, 433920, 434176, 434432, 434688, 434944, 435200, 436992, 439552, 441600, 442624, 444672, 445696, 445952, 446720, 447488, 447744, 448000, 448256, 453376, 453632, 453888, 603521, 132608, 500224, 34304, 114432, 479236, 497664, 554240, 606500, 181248, 183808, 192768, 196864, 238592, 393216, 6655, 416000, 416256, 514304, 483584, 122880, 139264, 149504, 150528, 478976, 484096, 26624, 27136, 30976, 36352, 130048, 130304, 137472, 166912, 167936, 171520, 266496, 266752, 267008, 267264, 267520, 267776, 289536, 513794, 500480, 12032, 63853, 34560, 168960, 170752, 169216, 12815, 13311, 13312, 13568, 13824, 168448, 169728, 54929, 500736, 23296, 27392, 35328, 36096, 126464, 126720, 126976, 127232, 127488, 127744, 128000, 128256, 128768, 129024, 129280, 130560, 132352, 134912, 135168, 135424, 142080, 208128, 484870, 130816, 29696, 48128, 33024, 479488, 226560, 22272, 163328, 210944, 392960, 178944, 220160, 220416, 279808, 316928, 317440, 317696, 318208, 325120, 360448, 360704, 360960, 361216, 368128, 370176, 388096, 443648, 443904, 254720, 254976, 263936, 264448, 297984, 298240, 299264, 505856, 200704, 393728, 393984, 394240, 451328, 451584, 483328, 500992, 284160, 284416, 284672, 284928, 357888, 358144, 32768, 34047, 200192, 275968, 343296, 344064, 344832, 347392, 347648, 348928, 479744, 33536, 39936, 49168, 49408, 22528, 501252, 22016, 282112, 355840, 200448, 219904, 268032, 268288, 355072, 355328, 441344, 10496, 379648, 384256, 444416, 391168, 391449, 38784, 255488, 257536, 265728, 136960, 192256, 204288, 253696, 253952, 254464, 265984, 266240, 280576, 303104, 356864, 357120, 375040, 375296, 436736, 454400, 24832, 188416, 262912, 263168, 263424, 263680, 351744, 353024, 341504, 341760, 261376, 287232, 480000, 114304, 184096, 185856, 195840, 199168, 204825, 205056, 205312, 206592, 207104, 441984, 442240, 442496, 447268, 485122, 2816, 141568, 160512, 225792, 226816, 227072, 227840, 233216, 233472, 275712, 282880, 285440, 285696, 286464, 297728, 304896, 318464, 318720, 318976, 319232, 319488, 319744, 320256, 320512, 320768, 321024, 321280, 321536, 321792, 322048, 322304, 322560, 322816, 323072, 323328, 323584, 323840, 324096, 324352, 326912, 327424, 327680, 333312, 335616, 336128, 336384, 342784, 349440, 349696, 387072, 387584, 387840, 395264, 418048, 426496, 430080, 430848, 431104, 431360, 431616, 431872, 444928, 453120, 485376, 312832, 355584, 367104, 370432, 242432, 245760, 253184, 279552, 301056, 301312, 301568, 301824, 302080, 302336, 302592, 317184, 361472, 361728, 361984, 362240, 362496, 366848, 388352, 392704, 412928, 413184, 413952, 414464, 414720, 417024, 417280, 23040, 26368, 27904, 148224, 150272, 157184, 157440, 174592, 174848, 222976, 250112, 258048, 258304, 258816, 259072, 259328, 259584, 260096, 260352, 264192, 282368, 282624, 364032, 373248, 485632, 14080, 16384, 25088, 26112, 34048, 34816, 37376, 48389, 128512, 138240, 139008, 139776, 140032, 143104, 143360, 144640, 144896, 145152, 147712, 147968, 148480, 148736, 149248, 150016, 155648, 156928, 161024, 166144, 166656, 168192, 175104, 175616, 177152, 207872, 208640, 223232, 223488, 226304, 229120, 229376, 230912, 231936, 232192, 232448, 232704, 232960, 233728, 233984, 234496, 235264, 235520, 235776, 303360, 313088, 313344, 313600, 313856, 314112, 315904, 316160, 327168, 328960, 340224, 349184, 349952, 350464, 350720, 350976, 351232, 375552, 380416, 380928, 381696, 382720, 383232, 384000, 384512, 384768, 385024, 385280, 385536, 385792, 389120, 394496, 395008, 396544, 396800, 399104, 399360, 402944, 403200, 403456, 403712, 403968, 410624, 418304, 418560, 424960, 425216, 425728, 425984, 437248, 438528, 438784, 449280, 449536, 449792, 450048, 450304, 450560, 450816, 451072, 147200, 260864, 268800, 283136, 283392, 283648, 283904, 358400, 358656, 358912, 359168, 359424, 359680, 359936, 369664, 412672, 417792, 439040, 439296, 452096, 12799, 22784, 36864, 141056, 142592, 144128, 154112, 176128, 177408, 212736, 342528, 386560, 386816, 485888, 11776, 155392, 242688, 242944, 243456, 243968, 244480, 244736, 245504, 246272, 249600, 249856, 258560, 298752, 372736, 19968, 228096, 228352, 228608, 228864, 231680, 269056, 273408, 308224, 311808, 312064, 312320, 312576, 314880, 315136, 315392, 315648, 320000, 325632, 325888, 332288, 332800, 333824, 334336, 335360, 336640, 337920, 338176, 338432, 338688, 338944, 339200, 339456, 339712, 339968, 340480, 342272, 343552, 343808, 344320, 344576, 345088, 345344, 345600, 345856, 346112, 346368, 346624, 346880, 347136, 347904, 348160, 348416, 348672, 351488, 363008, 363264, 363520, 364288, 364544, 365056, 365312, 366592, 374528, 375808, 378624, 380672, 381184, 381440, 388864, 397824, 401152, 402688, 416512, 425472, 429824, 430336, 15872, 487424, 23552, 48645, 131584, 131840, 134656, 136192, 151552, 151808, 296960, 297472, 26880, 29440, 129536, 129792, 136704, 138496, 140288, 145408, 145664, 155904, 167424, 168704, 169472, 169984, 173824, 174080, 174336, 203776, 208384, 210176, 224000, 224256, 224512, 224768, 225280, 236544, 236800, 237056, 237312, 237568, 237824, 238080, 238336, 241920, 244224, 247040, 247808, 248064, 250368, 250624, 250880, 251136, 251392, 259840, 260608, 261632, 261888, 262144, 262400, 264704, 264960, 265216, 265472, 281088, 281344, 281600, 281856, 287744, 297216, 303616, 303872, 304128, 304384, 304640, 305152, 305408, 305664, 305920, 306176, 306432, 306688, 306944, 307200, 307456, 307712, 307968, 308480, 308736, 308992, 309248, 314368, 314624, 317952, 325376, 326144, 370688, 372992, 397312, 397568, 402176, 476672, 28928, 133120, 157696, 191232, 241152, 328448, 328704, 362752, 364800, 372480, 377088, 377344, 377600, 377856, 378112, 378368, 378880, 563968, 133376, 133632, 133888, 136448, 137728, 149760, 152320, 163584, 163840, 164608, 165120, 166400, 173568, 190976, 208896, 213248, 223744, 227328, 227584, 234752, 235008, 239616, 256512, 256768, 257024, 257280, 273664, 273920, 274176, 274432, 274688, 274944, 275200, 275456, 280832, 288256, 290048, 291584, 291840, 292096, 293120, 293376, 294144, 299520, 369920, 371456, 397056, 404224, 404480, 404736, 404992, 405248, 405504, 405760, 406016, 406272, 406528, 406784, 407040, 407296, 407552, 407808, 408064, 408320, 408576, 408832, 409088, 409344, 409600, 409856, 413440, 415232, 415488, 417536, 430592, 437504, 437760, 438016, 438272, 440832, 132096, 140544, 142336, 145920, 146176, 146432, 146688, 146944, 147456, 150784, 151040, 151296, 156672, 196096, 199680, 221952, 239104, 268544, 272384, 272640, 272896, 273152, 276224, 276480, 309760, 310016, 310272, 310528, 310784, 311040, 311296, 311552, 324608, 326656, 327936, 328192, 329216, 329472, 329728, 329984, 330240, 330496, 330752, 331264, 331520, 331776, 332032, 332544, 333056, 333568, 334080, 334592, 334848, 335104, 336896, 337152, 337408, 337664, 343040, 350208, 371200, 374272, 376576, 379392, 380160, 381952, 382208, 382464, 382976, 383488, 383744, 387328, 394752, 395520, 399872, 402432, 416768, 429056, 440064, 440320, 440576, 445184, 445440, 446976, 448512, 448768, 449024, 37632, 152576, 153600, 158464, 210432, 210688, 212992, 226048, 244992, 252928, 255744, 309504, 316416, 316672, 365568, 365824, 366080, 366336, 376320, 389888, 426240, 432128, 432384, 444160, 452352, 48901, 441088, 16640, 28160, 28416, 29952, 36608, 122624, 140800, 141312, 143872, 148992, 154368, 159232, 159488, 164864, 165376, 167168, 167680, 175360, 176640, 176896, 179968, 211200, 211456, 219392, 219648, 234240, 246016, 248320, 248576, 248832, 249088, 249344, 252672, 256000, 256256, 257792, 262656, 287488, 299776, 300032, 300288, 357376, 357632, 368640, 368896, 369152, 369408, 370944, 373504, 373760, 389376, 391680, 391936, 392192, 392448, 395776, 401664, 414208, 415744, 443392, 452608, 452864, 14336, 14592, 29184, 30208, 137216, 142848, 155136, 159744, 160000, 160256, 187392, 187648, 207616, 240128, 242176, 247296, 253440, 254208, 277248, 288000, 296192, 302848, 356352, 356608, 376832, 401408, 411392, 411648, 411904, 412160, 418816, 419072, 419328, 419584, 419840, 420096, 420352, 420608, 420864, 421120, 421376, 421632, 421888, 422144, 422400, 422656, 422912, 423168, 423424, 423680, 423936, 424192, 424448, 424704, 426752, 429568, 435456, 435712, 16128, 31744, 38144, 38400, 131328, 134400, 138752, 139520, 143616, 144384, 152064, 152832, 153088, 153344, 153856, 154624, 154880, 156160, 158208, 158976, 170240, 175872, 176384, 178688, 206848, 209152, 209920, 212224, 212480, 213504, 213760, 214016, 214272, 214528, 214784, 215040, 215296, 215552, 215808, 216064, 216320, 216576, 216832, 217088, 217344, 217600, 217856, 218112, 218368, 218624, 218880, 219136, 222720, 243200, 245248, 247552, 251648, 251904, 252160, 252416, 255232, 277504, 277760, 278016, 278272, 278528, 278784, 279040, 279296, 280064, 280320, 286976, 288768, 289024, 289280, 294656, 294912, 295168, 295424, 295680, 295936, 296448, 296704, 352000, 352256, 352512, 352768, 353280, 353536, 353792, 354048, 354304, 354560, 354816, 360192, 367360, 379904, 386048, 386304, 390144, 396032, 396288, 398080, 398336, 398592, 398848, 442880, 443136, 446208, 446464, 454144, 491776, 141824, 157952, 158720, 246784, 298496, 324864, 335872, 340736, 340992, 341248, 342016, 374016, 374784, 379136, 393472, 410880, 413696, 435968, 172800, 211968, 243712, 42112, 42624, 44928, 45952, 131072, 221440, 221696, 222208, 222464, 285184, 288512, 289792, 299008, 390400, 390656, 390912, 399616, 400128, 400384, 240384, 240640, 240896, 241408, 241664, 246528, 412416, 414976, 436224, 436480, 507904, 24576, 132864, 137984, 156416, 196608, 371712, 371968, 372224, 376064, 410368, 480278, 876544, 24064, 77214, 83358, 84629, 509312, 527746, 537019, 537275, 594688, 603285, 605845, 606101, 606357, 844032, 844288, 844544, 844800, 855808, 520192, 552192, 856064, 10240, 60416, 60672, 60928, 480512, 492032, 567040, 855552, 857088, 857344, 857600, 857856, 475392, 855040, 855296, 497920, 516352, 516608, 516864, 42240, 480770, 867871, 868127, 868383, 498176, 856320, 856576, 10752, 498432, 476160, 515505, 515761, 516017, 907775, 914687, 44032, 914944, 908031, 492288, 912640, 915200, 916991, 913151, 911103, 913663, 913919, 914175, 915456, 916735, 917247, 914688, 912383, 912639, 913407, 911872, 907519, 11008, 915967, 916223, 907008, 907007, 908032, 914431, 916479, 911740, 482333, 482589, 482845, 910337, 910593, 911361, 498688, 492544, 45312, 497152, 487680, 492800, 481290, 919296, 17408, 17664, 45568, 487936, 491264, 45056, 4608, 8448, 8704, 8960, 9216, 9472, 9728, 9984, 475904, 484352, 39168, 256, 512, 1023, 1279, 1535, 1791, 3839, 5631, 100609, 117759, 124927, 499455, 503807, 504319, 504575, 504831, 505087, 513023, 513279, 513791, 556287, 565759, 586751, 671743, 868863, 869119, 869887, 870911, 871167, 871423, 871679, 871935, 872191, 872703, 872959, 873471, 873727, 873983, 874495, 875007, 875263, 875775, 876287, 876543, 885759, 886015, 886271, 886527, 886783, 887039, 887551, 887807, 888063, 888319, 888575, 888831, 889087, 889343, 889599, 889855, 890111, 890367, 890623, 890879, 891135, 891391, 891647, 891903, 892159, 892415, 892671, 892927, 893183, 893439, 893695, 893951, 894207, 894463, 894719, 894975, 895231, 895487, 895743, 895999, 896255, 896473, 896728, 896981, 897221, 897479, 897753, 898047, 898303, 898559, 898815, 899071, 899327, 899583, 899839, 900059, 900315, 900556, 900863, 901083, 901338, 901588, 901843, 902099, 902344, 902603, 902867, 903117, 903374, 903631, 903897, 904133, 904402, 904646, 904905, 905168, 905430, 905687, 905945, 906193, 906442, 906705, 908292, 908544, 908800, 909086, 909312, 909568, 909824, 910080, 911345]

In [ ]:
self.assertListEqual(tt, expected)

In [ ]:
nblock_used = 0

In [ ]:
tpos = bzd.tt[bzd.origptr] >> 8

In [ ]:
print(tpos)

In [ ]:
self.assertEqual(tpos, 94)

In [ ]:
tpos = bzd.tt[tpos]

In [ ]:
print(tpos)

In [ ]:
self.assertEqual(tpos, 68736)

In [ ]:
k0 = tpos & 0xff

In [ ]:
print(k0)

In [ ]:
self.assertEqual(k0, 128)

In [ ]:
tpos >>= 8

In [ ]:
print(tpos)

In [ ]:
self.assertEqual(tpos, 268)

In [ ]:
nblock_used += 1

In [ ]:
print(nblock_used)

In [ ]:
self.assertEqual(nblock_used, 1)

In [ ]:
def op1():
    if cs_avail_out == 0:
        return True
    if c_state_out_len == 1:
        return False
    cs_next_out.append(c_state_out_ch)
    c_state_out_len -= 1
    cs_avail_out -= 1

In [ ]:
def op2():
    if cs_avail_out == 0:
        c_state_out_len = 1
        return True
    cs_next_out.append(c_state_out_ch)
    cs_avail_out -= 1

In [ ]:
def op3():
    if nblock_used == s_save_nblockpp:
        c_state_out_len = 0
        return True
    c_state_out_ch = k0
    tpos = bzd.tt[tpos]
    k1 = tpos & 0xff
    tpos >>= 8
    nblock_used += 1

In [ ]:
def op4():
    if op2():
        return True
    if op3():
        return True
    if k1 != k0:
        k0 = k1
        if op4():
            return True
        return False
    if nblock_used == s_save_nblockpp:
        if op4():
            return True
        return False

    op6()

In [ ]:
def op6():
    c_state_out_len = 2
    tpos = bzd.tt[tpos]
    k1 = tpos & 0xff
    tpos >>= 8
    nblock_used += 1
    if nblock_used == s_save_nblockpp:
        return
    if k1 != k0:
        k0 = k1
        return

    c_state_out_len = 3
    tpos = bzd.tt[tpos]
    k1 = tpos & 0xff
    tpos >>= 8
    nblock_used += 1
    if nblock_used == s_save_nblockpp:
        return
    if k1 != k0:
        k0 = k1
        return

    tpos = bzd.tt[tpos]
    k1 = tpos & 0xff
    tpos >>= 8
    nblock_used += 1
    c_state_out_len = k1 + 4
    tpos = bzd.tt[tpos]
    k0 = tpos & 0xff
    tpos >>= 8
    nblock_used += 1

In [ ]:
def foo():
    while True:
        if c_state_out_len > 0:
            while True:
                match op1():
                    case False:
                        break
                    case True:
                        return
            if op2():
                return
        if op3():
            return
        if k1 != k0:
            k0 = k1
            if op4():
                return
            continue
        if nblock_used == s_save_nblockpp:
            if op4():
                return
            continue
    
        op6()

In [ ]:
class UnRLE:
    """Remove run length encoding from compressed data."""
    def __init__(self, avail_out, tt, nblock, nblock_used, k0, tpos):
        self.out_ch = 0
        self.out_len = 0

        self.nblock_used = nblock_used
        self.k0 = k0
        self.tpos = tpos
        self.tt = tt
        self.nblockpp = nblock + 1

        self.avail_out = avail_out
        self.next_out = bytearray()

        self.k1 = None

    def _op1(self):
        if self.avail_out == 0:
            return True
        if self.out_len == 1:
            return False
        self.next_out.append(self.out_ch)
        self.out_len -= 1
        self.avail_out -= 1
    
    def _op2(self):
        if self.avail_out == 0:
            self.out_len = 1
            return True
        self.next_out.append(self.out_ch)
        self.avail_out -= 1
    
    def _op3(self):
        if self.nblock_used == self.nblockpp:
            self.out_len = 0
            return True
        self.out_ch = self.k0
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
    
    def _op4(self):
        if self._op2():
            return True
        if self._op3():
            return True
        if self.k1 != self.k0:
            self.k0 = self.k1
            if self._op4():
                return True
            return False
        if self.nblock_used == self.nblockpp:
            if self._op4():
                return True
            return False
    
        self._op5()
    
    def _op5(self):
        self.out_len = 2
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        if self.nblock_used == self.nblockpp:
            return
        if self.k1 != self.k0:
            self.k0 = self.k1
            return
    
        self.out_len = 3
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        if self.nblock_used == self.nblockpp:
            return
        if self.k1 != self.k0:
            self.k0 = self.k1
            return
    
        self.tpos = self.tt[self.tpos]
        self.k1 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1
        self.out_len = self.k1 + 4
        self.tpos = self.tt[self.tpos]
        self.k0 = self.tpos & 0xff
        self.tpos >>= 8
        self.nblock_used += 1

    def run(self):
        while True:
            if self.out_len > 0:
                while True:
                    match self._op1():
                        case False:
                            break
                        case True:
                            return self.next_out
                if self._op2():
                    return self.next_out
            if self._op3():
                return self.next_out
            if self.k1 != self.k0:
                self.k0 = self.k1
                if self._op4():
                    return self.next_out
                continue
            if self.nblock_used == self.nblockpp:
                if self._op4():
                    return self.next_out
                continue
        
            self._op5()

        return self.next_out

In [ ]:
ur = UnRLE(0x1728, bzd.tt, nblock, nblock_used, k0, tpos)

In [ ]:
out = ur.run()

In [ ]:
sha256 = hashlib.sha256(out).hexdigest()

In [ ]:
self.assertEqual(sha256, 'eeaefe8c8a5d42855d886e5368d5c752a03d821681c7b2ceb5d900aa6cf70e18')

In [ ]:
uc = bzd._get_bits(8, 'BZ_X_ENDHDR_1')

In [ ]:
self.assertIs(uc, 0x17)

In [ ]:
if uc != 0x17:
    raise RuntimeError('BZ_DATA_ERROR: Incorrect BZ_X_ENDHDR_1')